# Creating a Working Dataset for Clustering

In this notebook, I prepare and refine the dataset I will use for clustering. I clean and merge the Urban Displacement Project's SCAG region dataset to LA Times neighborhood boundary data, isoalte our 6 neighborhods of interest (Westlake, Koreatown, Pico-Union, Downtown, Echo Park, and Silver Lake), and match census tracts with census data. I calculate percent change in household size, income, rent, percent of population with bachelor's degree and above, and percent of white population using census data, which are indicators used by the City of LA to determine displacement risk in this dataset:https://geohub.lacity.org/maps/70ed646893f642ddbca858c381471fa2. We update the City of LA's displacement risk indicators, looking at percent change between 2010 and 2020. 

## Merging Neighborhood Indicators and Neighborhood Boundary Data

First, I read in the Urban Displacement project gentrification typology data and merge it with LA neighborhood boundary data. This allows me to contextualize each census tract within a neighborhood. We start with 4553 census tracts across the SCAG region. After matching with the neighborhood boundary data, we are left with 1992 census tracts within the City of LA. 

In [810]:
import pandas as pd
import geopandas as gpd

df = gpd.read_file('scag-Copy1.gpkg')
df

,GEOID,Typology,geometry
0,6.037129e+09,Stable Moderate/Mixed Income,"MULTIPOLYGON (((-118.44870 34.16486, -118.4399..."
1,6.037132e+09,Stable Moderate/Mixed Income,"MULTIPOLYGON (((-118.55355 34.22046, -118.5461..."
2,6.037134e+09,At Risk of Becoming Exclusive,"MULTIPOLYGON (((-118.57103 34.21559, -118.5710..."
3,6.037134e+09,Advanced Gentrification,"MULTIPOLYGON (((-118.60590 34.21966, -118.6058..."
4,6.037242e+09,Low-Income/Susceptible to Displacement,"MULTIPOLYGON (((-118.25364 33.94328, -118.2491..."
...,...,...,...
4548,6.037920e+09,At Risk of Becoming Exclusive,"MULTIPOLYGON (((-118.47445 34.43330, -118.4683..."
4549,6.037542e+09,Low-Income/Susceptible to Displacement,"MULTIPOLYGON (((-118.19843 33.91057, -118.1987..."
4550,6.111004e+09,Ongoing Displacement,"MULTIPOLYGON (((-119.20831 34.18122, -119.2045..."
4551,6.065045e+09,Low-Income/Susceptible to Displacement,"MULTIPOLYGON (((-116.49298 33.81588, -116.4928..."


In [811]:
df.shape

(4553, 3)

In [812]:
type(df)

geopandas.geodataframe.GeoDataFrame

In [813]:
neighborhoods = gpd.read_file('LA_Times_Neighborhood_Boundaries (1).geojson')

In [814]:
join = gpd.sjoin(neighborhoods, df)
join

,OBJECTID,name,geometry,index_right,GEOID,Typology
0,1,Adams-Normandie,"POLYGON ((-118.30069 34.03731, -118.30388 34.0...",692,6.037222e+09,High Student Population
31,32,Exposition Park,"POLYGON ((-118.29155 34.01837, -118.28294 34.0...",692,6.037222e+09,High Student Population
93,94,University Park,"POLYGON ((-118.27986 34.01848, -118.28294 34.0...",692,6.037222e+09,High Student Population
0,1,Adams-Normandie,"POLYGON ((-118.30069 34.03731, -118.30388 34.0...",4033,6.037222e+09,Early/Ongoing Gentrification
0,1,Adams-Normandie,"POLYGON ((-118.30069 34.03731, -118.30388 34.0...",81,6.037222e+09,Advanced Gentrification
...,...,...,...,...,...,...
112,113,Woodland Hills,"POLYGON ((-118.57240 34.18668, -118.57161 34.1...",4256,6.037138e+09,Stable/Advanced Exclusive
112,113,Woodland Hills,"POLYGON ((-118.57240 34.18668, -118.57161 34.1...",1331,6.037137e+09,Stable Moderate/Mixed Income
112,113,Woodland Hills,"POLYGON ((-118.57240 34.18668, -118.57161 34.1...",669,6.037137e+09,Stable/Advanced Exclusive
112,113,Woodland Hills,"POLYGON ((-118.57240 34.18668, -118.57161 34.1...",33,6.037137e+09,At Risk of Becoming Exclusive


In [815]:
join.shape

(1992, 6)

Next, I filter the data for the 6 neighborhoods of interest: Westlake, Downtown, Echo Park, Silver Lake, Koreatown, and Pico-Union. I then reappend the 6 datasets together. 

In [816]:
Westlake = join.loc[join['name'] == 'Westlake']
Westlake.shape

(40, 6)

In [817]:
Westlake = Westlake.drop(['index_right'], axis=1)
Westlake

,OBJECTID,name,geometry,GEOID,Typology
106,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,Advanced Gentrification
106,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037210e+09,Advanced Gentrification
106,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification
106,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification
106,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,Low-Income/Susceptible to Displacement
106,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037211e+09,Early/Ongoing Gentrification
106,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037193e+09,Early/Ongoing Gentrification
106,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,At Risk of Gentrification
106,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,Ongoing Displacement
106,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037196e+09,Early/Ongoing Gentrification


In [818]:
Downtown = join.loc[join['name'] == 'Downtown']
Downtown.shape

(24, 6)

In [819]:
Downtown = Downtown.drop(['index_right'], axis=1)

In [820]:
Echo_Park = join.loc[join['name'] == 'Echo Park']
Echo_Park.shape

(17, 6)

In [821]:
Echo_Park = Echo_Park.drop(['index_right'], axis=1)

In [822]:
Silver_Lake = join.loc[join['name'] == 'Silver Lake']
Silver_Lake.shape

(26, 6)

In [823]:
Silver_Lake = Silver_Lake.drop(['index_right'], axis=1)

In [824]:
Koreatown = join.loc[join['name'] == 'Koreatown']
Koreatown.shape

(46, 6)

In [825]:
Koreatown = Koreatown.drop(['index_right'], axis=1)

In [826]:
Pico_Union = join.loc[join['name'] == 'Pico-Union']
Pico_Union.shape

(23, 6)

In [827]:
Pico_Union = Pico_Union.drop(['index_right'], axis=1)

In [830]:
join1 = Westlake.append(Downtown, ignore_index=True)
join1

/var/folders/bs/d9h__8tn0rb0h3qzd8bn64380000gn/T/ipykernel_16310/1328928621.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join1 = Westlake.append(Downtown, ignore_index=True)


,OBJECTID,name,geometry,GEOID,Typology
0,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,Advanced Gentrification
1,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037210e+09,Advanced Gentrification
2,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification
3,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification
4,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,Low-Income/Susceptible to Displacement
...,...,...,...,...,...
59,24,Downtown,"POLYGON ((-118.24450 34.06131, -118.24376 34.0...",6.037208e+09,At Risk of Gentrification
60,24,Downtown,"POLYGON ((-118.24450 34.06131, -118.24376 34.0...",6.037209e+09,Early/Ongoing Gentrification
61,24,Downtown,"POLYGON ((-118.24450 34.06131, -118.24376 34.0...",6.037208e+09,Advanced Gentrification
62,24,Downtown,"POLYGON ((-118.24450 34.06131, -118.24376 34.0...",6.037207e+09,Unavailable or Unreliable Data


In [831]:
join2 = join1.append(Echo_Park, ignore_index=True)

/var/folders/bs/d9h__8tn0rb0h3qzd8bn64380000gn/T/ipykernel_16310/3526225145.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join2 = join1.append(Echo_Park, ignore_index=True)


In [832]:
join3 = join2.append(Silver_Lake, ignore_index=True)

/var/folders/bs/d9h__8tn0rb0h3qzd8bn64380000gn/T/ipykernel_16310/2150512329.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join3 = join2.append(Silver_Lake, ignore_index=True)


In [833]:
join4 = join3.append(Koreatown, ignore_index=True)

/var/folders/bs/d9h__8tn0rb0h3qzd8bn64380000gn/T/ipykernel_16310/2650471741.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join4 = join3.append(Koreatown, ignore_index=True)


In [835]:
final_join = join4.append(Pico_Union, ignore_index=True)
final_join

/var/folders/bs/d9h__8tn0rb0h3qzd8bn64380000gn/T/ipykernel_16310/1279122901.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_join = join4.append(Pico_Union, ignore_index=True)


,OBJECTID,name,geometry,GEOID,Typology
0,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,Advanced Gentrification
1,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037210e+09,Advanced Gentrification
2,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification
3,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification
4,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,Low-Income/Susceptible to Displacement
...,...,...,...,...,...
171,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037221e+09,At Risk of Gentrification
172,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,Advanced Gentrification
173,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,At Risk of Gentrification
174,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,At Risk of Gentrification


After isolating the 6 neighborhoods of interest, we are left with 176 census tracts. 

## Census Data - Household Size

Next, I turn to the census data.I begin by reading in and cleaning household size data for all census tracts in LA from 2010.  

In [836]:
hh_size_2_2010 = pd.read_csv('ACSDT5Y2010.B25010_data_with_overlays_2022-05-27T173947.csv')
hh_size_2_2010

,GEO_ID,B25010_001E,B25010_001M,B25010_002E,B25010_002M,B25010_003E,B25010_003M,NAME
0,id,Estimate!!Total,Margin of Error!!Total,Estimate!!Total!!Owner occupied,Margin of Error!!Total!!Owner occupied,Estimate!!Total!!Renter occupied,Margin of Error!!Total!!Renter occupied,Geographic Area Name
1,1400000US06037101110,2.94,0.21,3.15,0.37,2.75,0.29,"Census Tract 1011.10, Los Angeles County, Cali..."
2,1400000US06037101122,2.81,0.22,2.90,0.24,2.14,0.74,"Census Tract 1011.22, Los Angeles County, Cali..."
3,1400000US06037101210,3.04,0.21,3.57,0.68,2.92,0.23,"Census Tract 1012.10, Los Angeles County, Cali..."
4,1400000US06037101220,2.72,0.22,3.21,0.55,2.40,0.33,"Census Tract 1012.20, Los Angeles County, Cali..."
...,...,...,...,...,...,...,...,...
2342,1400000US06037980031,-,**,-,**,-,**,"Census Tract 9800.31, Los Angeles County, Cali..."
2343,1400000US06037980033,-,**,-,**,-,**,"Census Tract 9800.33, Los Angeles County, Cali..."
2344,1400000US06037990100,-,**,-,**,-,**,"Census Tract 9901, Los Angeles County, California"
2345,1400000US06037990200,-,**,-,**,-,**,"Census Tract 9902, Los Angeles County, California"


In [837]:
hh_size_2_2010 = hh_size_2_2010.rename(columns={"B25010_001E": "Total_hhsize_2010", 
                                            "B25010_002E": "owner_occupied_hh_2010", 
                                            "B25010_003E": "renter_occupied_hh_2010"})

In [838]:
hh_size_2_2010 = hh_size_2_2010.drop(columns=['B25010_001M', 'B25010_002M', 'B25010_003M'])

In [840]:
hh_size_2_2010.drop(index=hh_size_2_2010.index[0], 
        axis=0, 
        inplace=True)

In [841]:
hh_size_2_2010['GEOID'] = hh_size_2_2010.GEO_ID.str.split('US0', 1, expand = True).get(1)
hh_size_2_2010['GEOID'] = hh_size_2_2010.GEOID.str.replace('NA','0').astype(float)

In [842]:
hh_size_2_2010['Total_hhsize_2010'] = hh_size_2_2010["Total_hhsize_2010"].str.replace("-","NaN")
hh_size_2_2010['owner_occupied_hh_2010'] = hh_size_2_2010["owner_occupied_hh_2010"].str.replace("-","NaN")
hh_size_2_2010['renter_occupied_hh_2010'] = hh_size_2_2010["renter_occupied_hh_2010"].str.replace("-","NaN")

In [843]:
hh_size_2_2010['Total_hhsize_2010'] = hh_size_2_2010.Total_hhsize_2010.astype(float)
hh_size_2_2010['owner_occupied_hh_2010'] = hh_size_2_2010.owner_occupied_hh_2010.astype(float)
hh_size_2_2010['renter_occupied_hh_2010'] = hh_size_2_2010.renter_occupied_hh_2010.astype(float)

I then do the same process with the 2020 data. 

In [844]:
hh_size_2_2020 = pd.read_csv('ACSDT5Y2020.B25010_data_with_overlays_2022-05-29T230437.csv')
hh_size_2_2020

,B25010_001E,B25010_001M,B25010_002E,B25010_002M,B25010_003E,B25010_003M,GEO_ID,NAME
0,Estimate!!Average household size --!!Total:,Margin of Error!!Average household size --!!To...,Estimate!!Average household size --!!Total:!!O...,Margin of Error!!Average household size --!!To...,Estimate!!Average household size --!!Total:!!R...,Margin of Error!!Average household size --!!To...,id,Geographic Area Name
1,2.54,0.21,2.56,0.28,2.52,0.37,1400000US06037101110,"Census Tract 1011.10, Los Angeles County, Cali..."
2,3.07,0.46,2.94,0.26,3.45,1.68,1400000US06037101122,"Census Tract 1011.22, Los Angeles County, Cali..."
3,2.64,0.32,2.90,0.70,2.45,0.28,1400000US06037101220,"Census Tract 1012.20, Los Angeles County, Cali..."
4,2.46,0.35,2.26,0.90,2.52,0.37,1400000US06037101221,"Census Tract 1012.21, Los Angeles County, Cali..."
...,...,...,...,...,...,...,...,...
2494,-,**,-,**,-,**,1400000US06037980038,"Census Tract 9800.38, Los Angeles County, Cali..."
2495,-,**,-,**,-,**,1400000US06037980039,"Census Tract 9800.39, Los Angeles County, Cali..."
2496,-,**,-,**,-,**,1400000US06037990100,"Census Tract 9901, Los Angeles County, California"
2497,-,**,-,**,-,**,1400000US06037990200,"Census Tract 9902, Los Angeles County, California"


In [845]:
hh_size_2_2020 = hh_size_2_2020.rename(columns={"B25010_001E": "Total_hhsize", 
                                            "B25010_002E": "owner_occupied_hh", 
                                            "B25010_003E": "renter_occupied_hh"})

In [846]:
hh_size_2_2020 = hh_size_2_2020.drop(columns=['B25010_001M', 'B25010_002M', 'B25010_003M'])

In [847]:
hh_size_2_2020.drop(index=hh_size_2_2020.index[0], 
        axis=0, 
        inplace=True)

In [848]:
hh_size_2_2020['GEOID'] = hh_size_2_2020.GEO_ID.str.split('US0', 1, expand = True).get(1)
hh_size_2_2020['GEOID'] = hh_size_2_2020.GEOID.str.replace('NA','0').astype(float)

In [849]:
hh_size_2_2020['Total_hhsize'] = hh_size_2_2020["Total_hhsize"].str.replace("-","NaN")
hh_size_2_2020['owner_occupied_hh'] = hh_size_2_2020["owner_occupied_hh"].str.replace("-","NaN")
hh_size_2_2020['renter_occupied_hh'] = hh_size_2_2020["renter_occupied_hh"].str.replace("-","NaN")

In [850]:
hh_size_2_2020['Total_hhsize'] = hh_size_2_2020.Total_hhsize.astype(float)
hh_size_2_2020['owner_occupied_hh'] = hh_size_2_2020.owner_occupied_hh.astype(float)
hh_size_2_2020['renter_occupied_hh'] = hh_size_2_2020.renter_occupied_hh.astype(float)

I merge the 2010, and then the 2020, household size data to my base dataset from above (which includes the census tracts, neighborhoods, and typologies for our neighborhoods of interest). All 176 census tracts from the base dataset match with the household size data. I do some final cleanup and then calculate percent change across the 2 years. 

In [851]:
final_join_hh_size = final_join.merge(
    hh_size_2_2010, 
    on=["GEOID"], 
    how="left",
    indicator=True,
)

In [852]:
final_join_hh_size.query("_merge != 'both'")
final_join_hh_size = final_join_hh_size.rename(columns={"_merge": "_merge1"})

In [853]:
final_join_hh_size = final_join_hh_size.merge(
    hh_size_2_2020, 
    on=["GEOID"], 
    how="left",
    indicator=True,
)
final_join_hh_size

,OBJECTID,name,geometry,GEOID,Typology,GEO_ID_x,Total_hhsize_2010,owner_occupied_hh_2010,renter_occupied_hh_2010,NAME_x,_merge1,Total_hhsize,owner_occupied_hh,renter_occupied_hh,GEO_ID_y,NAME_y,_merge
0,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,Advanced Gentrification,1400000US06037207710,1.49,1.48,1.49,"Census Tract 2077.10, Los Angeles County, Cali...",both,NaN,NaN,NaN,NaN,NaN,left_only
1,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037210e+09,Advanced Gentrification,1400000US06037210010,2.99,3.20,2.97,"Census Tract 2100.10, Los Angeles County, Cali...",both,2.72,2.77,2.72,1400000US06037210010,"Census Tract 2100.10, Los Angeles County, Cali...",both
2,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification,1400000US06037209300,2.73,4.47,2.68,"Census Tract 2093, Los Angeles County, California",both,2.35,2.30,2.35,1400000US06037209300,"Census Tract 2093, Los Angeles County, California",both
3,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification,1400000US06037209200,2.05,NaN,2.10,"Census Tract 2092, Los Angeles County, California",both,NaN,NaN,NaN,NaN,NaN,left_only
4,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,Low-Income/Susceptible to Displacement,1400000US06037208000,3.03,2.63,3.06,"Census Tract 2080, Los Angeles County, California",both,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037221e+09,At Risk of Gentrification,1400000US06037221110,3.01,2.96,3.01,"Census Tract 2211.10, Los Angeles County, Cali...",both,3.30,2.16,3.47,1400000US06037221110,"Census Tract 2211.10, Los Angeles County, Cali...",both
172,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,Advanced Gentrification,1400000US06037209820,4.06,5.26,3.78,"Census Tract 2098.20, Los Angeles County, Cali...",both,3.61,4.53,3.43,1400000US06037209820,"Census Tract 2098.20, Los Angeles County, Cali...",both
173,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,At Risk of Gentrification,1400000US06037209810,2.53,2.09,2.54,"Census Tract 2098.10, Los Angeles County, Cali...",both,2.86,2.20,2.87,1400000US06037209810,"Census Tract 2098.10, Los Angeles County, Cali...",both
174,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,At Risk of Gentrification,1400000US06037209520,3.28,5.11,3.09,"Census Tract 2095.20, Los Angeles County, Cali...",both,3.03,4.33,3.02,1400000US06037209520,"Census Tract 2095.20, Los Angeles County, Cali...",both


In [854]:
final_join_hh_size.query("_merge != 'both'")
final_join_hh_size = final_join_hh_size.rename(columns={"_merge": "_merge2"})

In [855]:
final_join_hh_size.columns

Index(['OBJECTID', 'name', 'geometry', 'GEOID', 'Typology', 'GEO_ID_x',
       'Total_hhsize_2010', 'owner_occupied_hh_2010',
       'renter_occupied_hh_2010', 'NAME_x', '_merge1', 'Total_hhsize',
       'owner_occupied_hh', 'renter_occupied_hh', 'GEO_ID_y', 'NAME_y',
       '_merge2'],
      dtype='object')

In [856]:
final_join_hh_size = final_join_hh_size.dropna(subset=['Total_hhsize_2010', 'owner_occupied_hh_2010', 'renter_occupied_hh_2010', 'Total_hhsize', 'owner_occupied_hh', 'renter_occupied_hh'])

In [857]:
final_join_hh_size['Total_hh_percent_change'] = (final_join_hh_size['Total_hhsize']-final_join_hh_size['Total_hhsize_2010'])/final_join_hh_size['Total_hhsize_2010']
final_join_hh_size['owner_occupied_percent_change'] = (final_join_hh_size['owner_occupied_hh']-final_join_hh_size['owner_occupied_hh_2010'])/final_join_hh_size['owner_occupied_hh_2010']
final_join_hh_size['renter_occupied_percent_change'] = (final_join_hh_size['renter_occupied_hh']-final_join_hh_size['renter_occupied_hh_2010'])/final_join_hh_size['renter_occupied_hh_2010']

I calculate summary statistics to observe high level trends. Overall, household size went down (by about 5%) over the 10 year period. Renter household size saw a larger percent decrease than owner households, which makes sense given that renter turnover is predictably higher than owner turnover. I am intrigued by the positive mean for owner occupied percent change. Perhaps there is variation across our neighborhoods of interest which is driving the discrepancy between the mean and median. 

In [858]:
final_join_hh_size['Total_hh_percent_change'].mean()

-0.05445347419914042

In [859]:
final_join_hh_size['Total_hh_percent_change'].median()

-0.05839416058394165

In [860]:
final_join_hh_size['owner_occupied_percent_change'].mean()

0.018548706599183777

In [861]:
final_join_hh_size['owner_occupied_percent_change'].median()

-0.06116207951070342

In [862]:
final_join_hh_size['renter_occupied_percent_change'].mean()

-0.060211608638048655

In [863]:
final_join_hh_size['renter_occupied_percent_change'].median()

-0.06484641638225254

I initally worked with a census dataset that did not provide a total average household size by census tract so calculated it by hand. We do not end up using these data in our final analyses but wanted to keep them here for reference. 

In [864]:
#hh_size_2020 = gpd.read_file('acs2020_5yr_B11016_14000US06037212701.geojson')
#hh_size_2020.head()

In [865]:
#hh_size_2020 = hh_size_2020.rename(columns={"B11016001": "Total", "B11016002": "Family_hh_total", 
 #                                           "B11016003": "2_pp_hh", "B11016004":"3_pp_hh", "B11016005": "4_pp_hh","B11016006":"5_pp_hh", "B11016007":"6_pp_hh", "B11016008":"7plus_pp_hh","B11016009":"Nonfamily_hh_total", "B11016010":"1_pp_hhnf", "B11016011":
  #                                         "2_pp_hhnf", "B11016012":"3_pp_hhnf", "B11016013":"4_pp_hhnf", "B11016014":
   #                                        "5_pp_hhnf", "B11016015":"6_pp_hhnf", "B11016016":"7plus_pp_hhnf"})

In [866]:
#hh_size_2020 = hh_size_2020.drop(columns=['B11016001, Error', 'B11016002, Error', 'B11016003, Error', 'B11016004, Error', 'B11016005, Error', 'B11016006, Error', 'B11016007, Error', 'B11016008, Error', 'B11016009, Error', 'B11016010, Error', 'B11016011, Error', 'B11016012, Error', 'B11016013, Error', 'B11016014, Error', 'B11016015, Error', 'B11016016, Error'])

In [867]:
#hh_size_2020['Family_hh_total_percent'] = hh_size_2020['Family_hh_total']/hh_size_2020['Total']
#hh_size_2020['2_pp_hh_percent'] = hh_size_2020['2_pp_hh']/hh_size_2020['Family_hh_total']
#hh_size_2020['3_pp_hh_percent'] = hh_size_2020['3_pp_hh']/hh_size_2020['Family_hh_total']
#hh_size_2020['4_pp_hh_percent'] = hh_size_2020['4_pp_hh']/hh_size_2020['Family_hh_total']
#hh_size_2020['5_pp_hh_percent'] = hh_size_2020['5_pp_hh']/hh_size_2020['Family_hh_total']
#hh_size_2020['6_pp_hh_percent'] = hh_size_2020['6_pp_hh']/hh_size_2020['Family_hh_total']
#hh_size_2020['7plus_pp_hh_percent'] = hh_size_2020['7plus_pp_hh']/hh_size_2020['Family_hh_total']
#hh_size_2020['Nonfamily_hh_total_percent'] = hh_size_2020['Nonfamily_hh_total']/hh_size_2020['Total']
#hh_size_2020['1_pp_hhnf_percent'] = hh_size_2020['1_pp_hhnf']/hh_size_2020['Nonfamily_hh_total']
#hh_size_2020['2_pp_hhnf_percent'] = hh_size_2020['2_pp_hhnf']/hh_size_2020['Nonfamily_hh_total']
#hh_size_2020['3_pp_hhnf_percent'] = hh_size_2020['3_pp_hhnf']/hh_size_2020['Nonfamily_hh_total']
#hh_size_2020['4_pp_hhnf_percent'] = hh_size_2020['4_pp_hhnf']/hh_size_2020['Nonfamily_hh_total']
#hh_size_2020['5_pp_hhnf_percent'] = hh_size_2020['5_pp_hhnf']/hh_size_2020['Nonfamily_hh_total']
#hh_size_2020['6_pp_hhnf_percent'] = hh_size_2020['6_pp_hhnf']/hh_size_2020['Nonfamily_hh_total']
#hh_size_2020['7plus_pp_hhnf_percent'] = hh_size_2020['7plus_pp_hhnf']/hh_size_2020['Nonfamily_hh_total']
#hh_size_2020.head()

In [868]:
#hh_size_2020['avg_size_family'] = ((2*hh_size_2020['2_pp_hh_percent'])+
 #                                  (3*hh_size_2020['3_pp_hh_percent'])+
  #                                 (4*hh_size_2020['4_pp_hh_percent'])+
   #                                (5*hh_size_2020['5_pp_hh_percent'])+
    #                               (6*hh_size_2020['6_pp_hh_percent'])+
     #                              (7*hh_size_2020['7plus_pp_hh_percent']))

In [869]:
#hh_size_2020['avg_size_nonfamily'] = ((1*hh_size_2020['1_pp_hhnf_percent'])+(2*hh_size_2020['2_pp_hhnf_percent'])+(3*hh_size_2020['3_pp_hhnf_percent'])+(4*hh_size_2020['4_pp_hhnf_percent'])+(5*hh_size_2020['5_pp_hhnf_percent'])+(6*hh_size_2020['6_pp_hhnf_percent'])+(7*hh_size_2020['7plus_pp_hhnf_percent']))

In [870]:
#hh_size_2020.head()

In [871]:
#hh_size_2010 = pd.read_csv('ACSDT5YSPT2010.B11016_data_with_overlays_2022-05-08T201304.csv')
#hh_size_2010.drop(index=hh_size_2010.index[0], 
 #       axis=0, 
  #      inplace=True)
#hh_size_2010

In [872]:
#hh_size_2010 = hh_size_2010.rename(columns={"GEO_ID":"geoid", "B11016_001E": "Total_2010", "B11016_002E": "Family_hh_total_2010", 
 #                                           "B11016_003E": "two_pp_hh_2010", "B11016_004E":"three_pp_hh_2010", "B11016_005E": "four_pp_hh_2010","B11016_006E":"five_pp_hh_2010", "B11016_007E":"six_pp_hh_2010", "B11016_008E":"sevenplus_pp_hh_2010","B11016_009E":"Nonfamily_hh_total_2010", "B11016_010E":"one_pp_hhnf_2010", "B11016_011E":
  #                                         "two_pp_hhnf_2010", "B11016_012E":"three_pp_hhnf_2010", "B11016_013E":"four_pp_hhnf_2010", "B11016_014E":
   #                                        "five_pp_hhnf_2010", "B11016_015E":"six_pp_hhnf_2010", "B11016_016E":"sevenplus_pp_hhnf_2010"})

In [873]:
#hh_size_2010

In [874]:
#hh_size_2010 = hh_size_2010.drop(columns=['POPGROUP', 'B11016_001M', 'B11016_002M', 'B11016_003M', 'B11016_004M', 'B11016_005M', 'B11016_006M', 'B11016_007M', 'B11016_008M', 'B11016_009M', 'B11016_010M', 'B11016_011M', 'B11016_012M', 'B11016_013M', 'B11016_014M', 'B11016_015M', 'B11016_016M'])

In [875]:
#hh_size_2010['Family_hh_total_2010'] = hh_size_2010.Family_hh_total_2010.astype(float)

In [876]:
#hh_size_2010['Total_2010'] = hh_size_2010.Total_2010.astype(float)
#hh_size_2010['Family_hh_total_2010'] = hh_size_2010.Family_hh_total_2010.astype(float)
#hh_size_2010['two_pp_hh_2010'] = hh_size_2010.two_pp_hh_2010.astype(float)
#hh_size_2010['three_pp_hh_2010'] = hh_size_2010.three_pp_hh_2010.astype(float)
#hh_size_2010['four_pp_hh_2010'] = hh_size_2010.four_pp_hh_2010.astype(float)
#hh_size_2010['five_pp_hh_2010'] = hh_size_2010.five_pp_hh_2010.astype(float)
#hh_size_2010['six_pp_hh_2010'] = hh_size_2010.six_pp_hh_2010.astype(float)
#hh_size_2010['sevenplus_pp_hh_2010'] = hh_size_2010.sevenplus_pp_hh_2010.astype(float)
#hh_size_2010['Nonfamily_hh_total_2010'] = hh_size_2010.Nonfamily_hh_total_2010.astype(float)
#hh_size_2010['one_pp_hhnf_2010'] = hh_size_2010.one_pp_hhnf_2010.astype(float)
#hh_size_2010['two_pp_hhnf_2010'] = hh_size_2010.two_pp_hhnf_2010.astype(float)
#hh_size_2010['three_pp_hhnf_2010'] = hh_size_2010.three_pp_hhnf_2010.astype(float)
#hh_size_2010['four_pp_hhnf_2010'] = hh_size_2010.four_pp_hhnf_2010.astype(float)
#hh_size_2010['five_pp_hhnf_2010'] = hh_size_2010.five_pp_hhnf_2010.astype(float)
#hh_size_2010['six_pp_hhnf_2010'] = hh_size_2010.six_pp_hhnf_2010.astype(float)
#hh_size_2010['sevenplus_pp_hhnf_2010'] = hh_size_2010.sevenplus_pp_hhnf_2010.astype(float)

In [877]:
#hh_size_2010['Family_hh_total_2010_percent'] = hh_size_2010['Family_hh_total_2010']/hh_size_2010['Total_2010']
#hh_size_2010['two_pp_hh_2010_percent'] = hh_size_2010['two_pp_hh_2010']/hh_size_2010['Family_hh_total_2010']
#hh_size_2010['three_pp_hh_2010_percent'] = hh_size_2010['three_pp_hh_2010']/hh_size_2010['Family_hh_total_2010']
#hh_size_2010['four_pp_hh_2010_percent'] = hh_size_2010['four_pp_hh_2010']/hh_size_2010['Family_hh_total_2010']
#hh_size_2010['five_pp_hh_2010_percent'] = hh_size_2010['five_pp_hh_2010']/hh_size_2010['Family_hh_total_2010']
#hh_size_2010['six_pp_hh_2010_percent'] = hh_size_2010['six_pp_hh_2010']/hh_size_2010['Family_hh_total_2010']
#hh_size_2010['sevenplus_pp_hh_2010_percent'] = hh_size_2010['sevenplus_pp_hh_2010']/hh_size_2010['Family_hh_total_2010']
#hh_size_2010['Nonfamily_hh_total_2010_percent'] = hh_size_2010['Nonfamily_hh_total_2010']/hh_size_2020['Total']
#hh_size_2010['one_pp_hhnf_2010_percent'] = hh_size_2010['one_pp_hhnf_2010']/hh_size_2010['Nonfamily_hh_total_2010']
#hh_size_2010['two_pp_hhnf_2010_percent'] = hh_size_2010['two_pp_hhnf_2010']/hh_size_2010['Nonfamily_hh_total_2010']
#hh_size_2010['three_pp_hhnf_2010_percent'] = hh_size_2010['three_pp_hhnf_2010']/hh_size_2010['Nonfamily_hh_total_2010']
#hh_size_2010['four_pp_hhnf_2010_percent'] = hh_size_2010['four_pp_hhnf_2010']/hh_size_2010['Nonfamily_hh_total_2010']
#hh_size_2010['five_pp_hhnf_2010_percent'] = hh_size_2010['five_pp_hhnf_2010']/hh_size_2010['Nonfamily_hh_total_2010']
#hh_size_2010['six_pp_hhnf_2010_percent'] = hh_size_2010['six_pp_hhnf_2010']/hh_size_2010['Nonfamily_hh_total_2010']
#hh_size_2010['sevenplus_pp_hhnf_2010_percent'] = hh_size_2010['sevenplus_pp_hhnf_2010']/hh_size_2010['Nonfamily_hh_total_2010']
#hh_size_2010.head()

In [878]:
#hh_size_2010['avg_size_family_2010'] = ((2*hh_size_2010['two_pp_hh_2010_percent'])+(3*hh_size_2010['three_pp_hh_2010_percent'])+
 #                                       (4*hh_size_2010['four_pp_hh_2010_percent'])+(5*hh_size_2010['five_pp_hh_2010_percent'])+(6*hh_size_2010['six_pp_hh_2010_percent'])+(7*hh_size_2010['sevenplus_pp_hh_2010_percent']))

In [879]:
#hh_size_2010['avg_size_nonfamily_2010'] = ((2*hh_size_2010['two_pp_hhnf_2010_percent'])+(3*hh_size_2010['three_pp_hhnf_2010_percent'])+(4*hh_size_2010['four_pp_hhnf_2010_percent'])+(5*hh_size_2010['five_pp_hhnf_2010_percent'])+(6*hh_size_2010['six_pp_hhnf_2010_percent'])+(7*hh_size_2010['sevenplus_pp_hhnf_2010_percent']))

In [880]:
#hh_size_2020['GEOID'] = hh_size_2020.geoid.str.split('US0', 1, expand = True).get(1)
#hh_size_2020['GEOID'] = hh_size_2020.GEOID.str.replace('NA','0').astype(float)

In [881]:
#hh_size_2010['GEOID'] = hh_size_2010.geoid.str.split('US0', 1, expand = True).get(1)
#hh_size_2010['GEOID'] = hh_size_2010.GEOID.str.replace('NA','0').astype(float)

In [882]:
#final_join_hh_size = final_join.merge(
 #   hh_size_2010, 
  #  on=["GEOID"], 
   # how="left",
    #indicator=True,
#)
#final_join_hh_size

In [883]:
#final_join_hh_size.query("_merge != 'both'")
#final_join_hh_size = final_join_hh_size.rename(columns={"_merge": "_merge1"})

In [884]:
#final_join_hh_size = final_join_hh_size.merge(
 #   hh_size_2020, 
  #  on=["GEOID"], 
   # how="left",
    #indicator=True,
#)
#final_join_hh_size

In [885]:
#final_join_hh_size.query("_merge != 'both'")
#final_join_hh_size = final_join_hh_size.rename(columns={"_merge": "_merge2"})

In [886]:
#final_join_hh_size.columns

In [887]:
#final_join_hh_size = final_join_hh_size.dropna(subset=['avg_size_family', 'avg_size_nonfamily', 'avg_size_family_2010', 'avg_size_nonfamily_2010'])

In [888]:
#final_join_hh_size['family_size_percent_change'] = (final_join_hh_size['avg_size_family']-final_join_hh_size['avg_size_family_2010'])/final_join_hh_size['avg_size_family_2010']
#final_join_hh_size['nonfamily_size_percent_change'] = (final_join_hh_size['avg_size_nonfamily']-final_join_hh_size['avg_size_nonfamily_2010'])/final_join_hh_size['avg_size_nonfamily_2010']

In [889]:
#final_join_hh_size['family_size_percent_change'].mean()

In [890]:
#final_join_hh_size['family_size_percent_change'].median()

In [891]:
#final_join_hh_size['nonfamily_size_percent_change'].mean()

In [892]:
#final_join_hh_size['nonfamily_size_percent_change'].median()

## Census Data - Gross Rent 

In this section, I read in and clean gross rent data for all census tracts in LA from 2020. Like with household size, I follow the same process for the 2010 data.   

In [893]:
rent_2020 = pd.read_csv('ACSDT5Y2020.B25064_data_with_overlays_2022-05-17T185744.csv')
rent_2020.drop(index=rent_2020.index[0], 
        axis=0, 
        inplace=True)
rent_2020.columns

Index(['B25064_001E', 'B25064_001M', 'GEO_ID', 'NAME'], dtype='object')

In [894]:
rent_2020 = rent_2020.rename(columns={'B25064_001E':'median_gross_rent'})

In [895]:
rent_2020 = rent_2020.drop(columns=['B25064_001M'])

In [896]:
rent_2020 = rent_2020.replace({'3,500+': '3500'})

In [897]:
rent_2020['median_gross_rent'] = rent_2020.median_gross_rent.str.replace('-','NaN').astype(float)

In [898]:
rent_2020['GEOID'] = rent_2020.GEO_ID.str.split('US0', 1, expand = True).get(1)
rent_2020['GEOID'] = rent_2020.GEOID.str.replace('NA','0').astype(float)

In [899]:
rent_2010 = pd.read_csv('ACSDT5Y2010.B25064_data_with_overlays_2022-05-17T191055.csv')
rent_2010.drop(index=rent_2010.index[0], 
        axis=0, 
        inplace=True)
rent_2010.columns

Index(['GEO_ID', 'B25064_001E', 'B25064_001M', 'NAME'], dtype='object')

In [900]:
rent_2010 = rent_2010.rename(columns={'B25064_001E':'median_gross_rent_2010'})

In [901]:
rent_2010 = rent_2010.drop(columns=['B25064_001M'])

In [902]:
rent_2010 = rent_2010.replace({'2,000+': '2000'})

In [903]:
rent_2010['median_gross_rent_2010'] = rent_2010.median_gross_rent_2010.str.replace('-','NaN').astype(float)

In [904]:
rent_2010['GEOID'] = rent_2010.GEO_ID.str.split('US0', 1, expand = True).get(1)
rent_2010['GEOID'] = rent_2010.GEOID.str.replace('NA','0').astype(float)

I merge the 2010, and then the 2020, gross rent data to my base dataset from above (which includes the census tracts, neighborhoods, typologies for our neighborhoods of interest, and household size percent change). Only 133 out of 176 rows match during this merge. I assume this is due to missing data. I move forward with the 133 rows (which remain constant for the future merges) I do some final cleanup and then calculate percent change across the 2 years. 

In [905]:
census_neighborhoods_UDP = final_join_hh_size.merge(
    rent_2020, 
    on=["GEOID"], 
    how="left",
    indicator=True,
)

In [906]:
census_neighborhoods_UDP.query("_merge1 != 'both'")
census_neighborhoods_UDP = census_neighborhoods_UDP.drop(columns=['GEO_ID', 'NAME_x', 'NAME_y', '_merge2'])
census_neighborhoods_UDP.columns

Index(['OBJECTID', 'name', 'geometry', 'GEOID', 'Typology', 'GEO_ID_x',
       'Total_hhsize_2010', 'owner_occupied_hh_2010',
       'renter_occupied_hh_2010', '_merge1', 'Total_hhsize',
       'owner_occupied_hh', 'renter_occupied_hh', 'GEO_ID_y',
       'Total_hh_percent_change', 'owner_occupied_percent_change',
       'renter_occupied_percent_change', 'median_gross_rent', 'NAME',
       '_merge'],
      dtype='object')

In [907]:
census_neighborhoods_UDP = census_neighborhoods_UDP.merge(
    rent_2010, 
    on=["GEOID"], 
    how="left",
    #indicator=True,
)
census_neighborhoods_UDP

,OBJECTID,name,geometry,GEOID,Typology,GEO_ID_x,Total_hhsize_2010,owner_occupied_hh_2010,renter_occupied_hh_2010,_merge1,...,GEO_ID_y,Total_hh_percent_change,owner_occupied_percent_change,renter_occupied_percent_change,median_gross_rent,NAME_x,_merge,GEO_ID,median_gross_rent_2010,NAME_y
0,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037210e+09,Advanced Gentrification,1400000US06037210010,2.99,3.20,2.97,both,...,1400000US06037210010,-0.090301,-0.134375,-0.084175,906.0,"Census Tract 2100.10, Los Angeles County, Cali...",both,1400000US06037210010,596.0,"Census Tract 2100.10, Los Angeles County, Cali..."
1,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification,1400000US06037209300,2.73,4.47,2.68,both,...,1400000US06037209300,-0.139194,-0.485459,-0.123134,1051.0,"Census Tract 2093, Los Angeles County, California",both,1400000US06037209300,827.0,"Census Tract 2093, Los Angeles County, California"
2,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037211e+09,Early/Ongoing Gentrification,1400000US06037211122,2.55,6.60,2.53,both,...,1400000US06037211122,0.003922,-0.431818,0.011858,1587.0,"Census Tract 2111.22, Los Angeles County, Cali...",both,1400000US06037211122,1099.0,"Census Tract 2111.22, Los Angeles County, Cali..."
3,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037193e+09,Early/Ongoing Gentrification,1400000US06037192700,2.80,2.60,2.83,both,...,1400000US06037192700,0.060714,0.219231,0.038869,1431.0,"Census Tract 1927, Los Angeles County, California",both,1400000US06037192700,906.0,"Census Tract 1927, Los Angeles County, California"
4,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,At Risk of Gentrification,1400000US06037208302,2.90,4.32,2.77,both,...,1400000US06037208302,-0.051724,-0.504630,0.028881,971.0,"Census Tract 2083.02, Los Angeles County, Cali...",both,1400000US06037208302,713.0,"Census Tract 2083.02, Los Angeles County, Cali..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037221e+09,Low-Income/Susceptible to Displacement,1400000US06037221120,4.55,7.29,4.10,both,...,1400000US06037221120,-0.142857,-0.443073,-0.051220,1152.0,"Census Tract 2211.20, Los Angeles County, Cali...",both,1400000US06037221120,967.0,"Census Tract 2211.20, Los Angeles County, Cali..."
129,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037221e+09,At Risk of Gentrification,1400000US06037221110,3.01,2.96,3.01,both,...,1400000US06037221110,0.096346,-0.270270,0.152824,994.0,"Census Tract 2211.10, Los Angeles County, Cali...",both,1400000US06037221110,658.0,"Census Tract 2211.10, Los Angeles County, Cali..."
130,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,Advanced Gentrification,1400000US06037209820,4.06,5.26,3.78,both,...,1400000US06037209820,-0.110837,-0.138783,-0.092593,980.0,"Census Tract 2098.20, Los Angeles County, Cali...",both,1400000US06037209820,782.0,"Census Tract 2098.20, Los Angeles County, Cali..."
131,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,At Risk of Gentrification,1400000US06037209810,2.53,2.09,2.54,both,...,1400000US06037209810,0.130435,0.052632,0.129921,900.0,"Census Tract 2098.10, Los Angeles County, Cali...",both,1400000US06037209810,617.0,"Census Tract 2098.10, Los Angeles County, Cali..."


In [908]:
census_neighborhoods_UDP = census_neighborhoods_UDP.drop(columns=['GEO_ID', 'NAME_y','_merge'])

In [910]:
census_neighborhoods_UDP['rent_percent_change'] = (census_neighborhoods_UDP
                                                   ['median_gross_rent']-census_neighborhoods_UDP
                                                   ['median_gross_rent_2010'])/census_neighborhoods_UDP['median_gross_rent_2010']






In [911]:
census_neighborhoods_UDP.rent_percent_change

0      0.520134
1      0.270859
2      0.444040
3      0.579470
4      0.361851
         ...   
128    0.191313
129    0.510638
130    0.253197
131    0.458671
132    0.305128
Name: rent_percent_change, Length: 133, dtype: float64

I calculate summary statistics to understand how rents have changed over the 10 year period. Without adjusting for inflation, rents increase by an average of 46%. This is very high. I figured there would be a notable increase, but 46% seems questionably high. In the next section, I adjust for inflation to see if the numbers make more sense.

In [912]:
census_neighborhoods_UDP['rent_percent_change'].mean()

0.4628410985455908

In [913]:
census_neighborhoods_UDP['rent_percent_change'].median()

0.4258849557522124

### Adjusting for inflation

To calculate inflation, I divide the median gross rent (2010) by the rental index for January 2010 and then multiply by the rental index for January 2020. I used the CPI for US City average from this website: https://fred.stlouisfed.org/release/tables?rid=10&eid=34483&od=2020-01-01#. I then merge this new column back to my base dataset and recalculate percent change. 

In [914]:
rent_2010['median_gross_rent_2010_inflation'] = (rent_2010['median_gross_rent_2010']/248.721)*337.570

In [915]:
rent_2010 = rent_2010.drop(columns={'GEO_ID', 'median_gross_rent_2010', 'NAME'})

In [916]:
census_neighborhoods_UDP = census_neighborhoods_UDP.merge(
    rent_2010, 
    on=["GEOID"], 
    how="left",
    indicator=True,
)
census_neighborhoods_UDP

,OBJECTID,name,geometry,GEOID,Typology,GEO_ID_x,Total_hhsize_2010,owner_occupied_hh_2010,renter_occupied_hh_2010,_merge1,...,GEO_ID_y,Total_hh_percent_change,owner_occupied_percent_change,renter_occupied_percent_change,median_gross_rent,NAME_x,median_gross_rent_2010,rent_percent_change,median_gross_rent_2010_inflation,_merge
0,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037210e+09,Advanced Gentrification,1400000US06037210010,2.99,3.20,2.97,both,...,1400000US06037210010,-0.090301,-0.134375,-0.084175,906.0,"Census Tract 2100.10, Los Angeles County, Cali...",596.0,0.520134,808.905239,both
1,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification,1400000US06037209300,2.73,4.47,2.68,both,...,1400000US06037209300,-0.139194,-0.485459,-0.123134,1051.0,"Census Tract 2093, Los Angeles County, California",827.0,0.270859,1122.423881,both
2,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037211e+09,Early/Ongoing Gentrification,1400000US06037211122,2.55,6.60,2.53,both,...,1400000US06037211122,0.003922,-0.431818,0.011858,1587.0,"Census Tract 2111.22, Los Angeles County, Cali...",1099.0,0.444040,1491.588688,both
3,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037193e+09,Early/Ongoing Gentrification,1400000US06037192700,2.80,2.60,2.83,both,...,1400000US06037192700,0.060714,0.219231,0.038869,1431.0,"Census Tract 1927, Los Angeles County, California",906.0,0.579470,1229.644541,both
4,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,At Risk of Gentrification,1400000US06037208302,2.90,4.32,2.77,both,...,1400000US06037208302,-0.051724,-0.504630,0.028881,971.0,"Census Tract 2083.02, Los Angeles County, Cali...",713.0,0.361851,967.700395,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037221e+09,Low-Income/Susceptible to Displacement,1400000US06037221120,4.55,7.29,4.10,both,...,1400000US06037221120,-0.142857,-0.443073,-0.051220,1152.0,"Census Tract 2211.20, Los Angeles County, Cali...",967.0,0.191313,1312.435178,both
129,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037221e+09,At Risk of Gentrification,1400000US06037221110,3.01,2.96,3.01,both,...,1400000US06037221110,0.096346,-0.270270,0.152824,994.0,"Census Tract 2211.10, Los Angeles County, Cali...",658.0,0.510638,893.053100,both
130,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,Advanced Gentrification,1400000US06037209820,4.06,5.26,3.78,both,...,1400000US06037209820,-0.110837,-0.138783,-0.092593,980.0,"Census Tract 2098.20, Los Angeles County, Cali...",782.0,0.253197,1061.348821,both
131,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,At Risk of Gentrification,1400000US06037209810,2.53,2.09,2.54,both,...,1400000US06037209810,0.130435,0.052632,0.129921,900.0,"Census Tract 2098.10, Los Angeles County, Cali...",617.0,0.458671,837.406934,both


In [917]:
census_neighborhoods_UDP['rent_percent_change_inflation'] = (census_neighborhoods_UDP['median_gross_rent']-census_neighborhoods_UDP['median_gross_rent_2010_inflation'])/census_neighborhoods_UDP['median_gross_rent_2010_inflation']

In [918]:
census_neighborhoods_UDP.rent_percent_change_inflation

0      0.120032
1     -0.063634
2      0.063966
3      0.163751
4      0.003410
         ...   
128   -0.122242
129    0.113036
130   -0.076647
131    0.074746
132   -0.038384
Name: rent_percent_change_inflation, Length: 133, dtype: float64

After adjusting for inflation, the percent change in rents makes a lot more sense. Inflation-adjusted rents increased by an average of 8% (as opposed to 46%). 

In [919]:
census_neighborhoods_UDP['rent_percent_change_inflation'].mean()

0.07781882534395211

In [920]:
census_neighborhoods_UDP['rent_percent_change_inflation'].median()

0.050589602392529125

Like with the household size data, I initally worked with a census dataset that did not provide total median gross rents by census tract. I initially calculated average rents by rental amounts. We do not end up using these data in our final analyses but wanted to keep them here for reference. 

In [921]:
#rent_2020 = pd.read_csv('ACSDT5Y2020.B25063_data_with_overlays_2022-04-28T181825.csv')
#rent_2020.drop(index=hh_size_2020.index[0], 
 #       axis=0, 
 #       inplace=True)
#rent_2020.columns

In [922]:
#rent_2020 = rent_2020.rename(columns={"B25063_001E": "Total", "B25063_002E": "Total_cash", 
#                                            "B25063_003E": "less_than_100", "B25063_004E":"onehundred_149", "B25063_005E": "onefifty_199","B25063_006E":"twohundred_249", "B25063_007E":"twofifty_299", "B25063_008E":"threehundred_349","B25063_009E":"threefifty_399", "B25063_010E":"fourhundred_449", "B25063_011E":
 #                                          "fourfifty_499", "B25063_012E":"fivehundred_549", "B25063_013E":"fivefifty_599", "B25063_014E":
  #                                         "sixhundred_649", "B25063_015E":"sixfifty_699", "B25063_016E":"sevenhundred_749", "B25063_017E":"sevenfifty_799",
   #                                        "B25063_018E":"eighthundred_899", "B25063_019E":"ninehundred_999", "B25063_020E":"onethousand_1249", "B25063_021E":"onethousandtwofifty_1499",
    #                                       "B25063_022E":"onethousandfive_1999", "B25063_023E":"twothousand_2499", "B25063_024E":"twothousandfive_2999",
     #                                      "B25063_025E":"threethousand_3499", "B25063_026E":"threethousandfiveplus", 
      #                                     "B25063_027E":"Total_no_cash"})

In [923]:
#rent_2020 = rent_2020.drop(columns=['B25063_001M', 'B25063_002M', 'B25063_003M', 'B25063_004M', 'B25063_005M',
 #                                   'B25063_006M', 'B25063_007M', 'B25063_008M','B25063_009M', 'B25063_010M', 'B25063_011M',
  #                                  'B25063_012M', 'B25063_013M', 'B25063_014M', 'B25063_015M', 'B25063_016M', 'B25063_017M',
   #                                 'B25063_018M', 'B25063_019M', 'B25063_020M', 'B25063_021M', 'B25063_022M', 'B25063_023M',
    #                                'B25063_024M', 'B25063_025M', 'B25063_026M', 'B25063_027M'])

In [924]:
#rent_2020['Total'] = rent_2020.Total.astype(float)
#rent_2020['Total_cash'] = rent_2020.Total_cash.astype(float)
#rent_2020['less_than_100'] = rent_2020.less_than_100.astype(float)
#rent_2020['onehundred_149'] = rent_2020.onehundred_149.astype(float)
#rent_2020['onefifty_199'] = rent_2020.onefifty_199.astype(float)
#rent_2020['twohundred_249'] = rent_2020.twohundred_249.astype(float)
#rent_2020['twofifty_299'] = rent_2020.twofifty_299.astype(float)
#rent_2020['threehundred_349'] = rent_2020.threehundred_349.astype(float)
#rent_2020['threefifty_399'] = rent_2020.threefifty_399.astype(float)
#rent_2020['fourhundred_449'] = rent_2020.fourhundred_449.astype(float)
#rent_2020['fourfifty_499'] = rent_2020.fourfifty_499.astype(float)
#rent_2020['fivehundred_549'] = rent_2020.fivehundred_549.astype(float)
#rent_2020['fivefifty_599'] = rent_2020.fivefifty_599.astype(float)
#rent_2020['sixhundred_649'] = rent_2020.sixhundred_649.astype(float)
#rent_2020['sixfifty_699'] = rent_2020.sixfifty_699.astype(float)
#rent_2020['sevenhundred_749'] = rent_2020.sevenhundred_749.astype(float)
#rent_2020['sevenfifty_799'] = rent_2020.sevenfifty_799.astype(float)
#rent_2020['eighthundred_899'] = rent_2020.eighthundred_899.astype(float)
#rent_2020['ninehundred_999'] = rent_2020.ninehundred_999.astype(float)
#rent_2020['onethousand_1249'] = rent_2020.onethousand_1249.astype(float)
#rent_2020['onethousandtwofifty_1499'] = rent_2020.onethousandtwofifty_1499.astype(float)
#rent_2020['onethousandfive_1999'] = rent_2020.onethousandfive_1999.astype(float)
#rent_2020['twothousand_2499'] = rent_2020.twothousand_2499.astype(float)
#rent_2020['twothousandfive_2999'] = rent_2020.twothousandfive_2999.astype(float)
#rent_2020['threethousand_3499'] = rent_2020.threethousand_3499.astype(float)
#rent_2020['threethousandfiveplus'] = rent_2020.threethousandfiveplus.astype(float)
#rent_2020['Total_no_cash'] = rent_2020.Total_no_cash.astype(float)

In [925]:
#rent_2020

In [926]:
#rent_2020['Total_cash_percent'] = rent_2020['Total_cash']/rent_2020['Total']
#rent_2020['less_than_100_percent'] = rent_2020['less_than_100']/rent_2020['Total_cash']
#rent_2020['onehundred_149_percent'] = rent_2020['onehundred_149']/rent_2020['Total_cash']
#rent_2020['onefifty_199_percent'] = rent_2020['onefifty_199']/rent_2020['Total_cash']
#rent_2020['twohundred_249_percent'] = rent_2020['twohundred_249']/rent_2020['Total_cash']
#rent_2020['twofifty_299_percent'] = rent_2020['twofifty_299']/rent_2020['Total_cash']
#rent_2020['threehundred_349_percent'] = rent_2020['threehundred_349']/rent_2020['Total_cash']
#rent_2020['threefifty_399_percent'] = rent_2020['threefifty_399']/rent_2020['Total_cash']
#rent_2020['fourhundred_449_percent'] = rent_2020['fourhundred_449']/rent_2020['Total_cash']
#rent_2020['fourfifty_499_percent'] = rent_2020['fourfifty_499']/rent_2020['Total_cash']
#rent_2020['fivehundred_549_percent'] = rent_2020['fivehundred_549']/rent_2020['Total_cash']
#rent_2020['fivefifty_599_percent'] = rent_2020['fivefifty_599']/rent_2020['Total_cash']
#rent_2020['sixhundred_649_percent'] = rent_2020['sixhundred_649']/rent_2020['Total_cash']
#rent_2020['sixfifty_699_percent'] = rent_2020['sixfifty_699']/rent_2020['Total_cash']
#rent_2020['sevenhundred_749_percent'] = rent_2020['sevenhundred_749']/rent_2020['Total_cash']
#rent_2020['sevenfifty_799_percent'] = rent_2020['sevenfifty_799']/rent_2020['Total_cash']
#rent_2020['eighthundred_899_percent'] = rent_2020['eighthundred_899']/rent_2020['Total_cash']
#rent_2020['ninehundred_999_percent'] = rent_2020['ninehundred_999']/rent_2020['Total_cash']
#rent_2020['onethousand_1249_percent'] = rent_2020['onethousand_1249']/rent_2020['Total_cash']
#rent_2020['onethousandtwofifty_1499_percent'] = rent_2020['onethousandtwofifty_1499']/rent_2020['Total_cash']
#rent_2020['onethousandfive_1999_percent'] = rent_2020['onethousandfive_1999']/rent_2020['Total_cash']
#rent_2020['twothousand_2499_percent'] = rent_2020['twothousand_2499']/rent_2020['Total_cash']
#rent_2020['twothousandfive_2999_percent'] = rent_2020['twothousandfive_2999']/rent_2020['Total_cash']
#rent_2020['threethousand_3499_percent'] = rent_2020['threethousand_3499']/rent_2020['Total_cash']
#rent_2020['threethousandfiveplus_percent'] = rent_2020['threethousandfiveplus']/rent_2020['Total_cash']
#rent_2020['Total_no_cash_percent'] = rent_2020['Total_no_cash']/rent_2020['Total']

In [927]:
#rent_2020['avg_rent_2020'] = ((50*rent_2020['less_than_100_percent'])+
 #                                       (124.5*rent_2020['onehundred_149_percent'])+
  #                                      (174.5*rent_2020['onefifty_199_percent'])+
   #                                     (224.5*rent_2020['twohundred_249_percent'])+
    #                                    (324.5*rent_2020['threehundred_349_percent'])+
     #                                   (374.5*rent_2020['threefifty_399_percent'])+
      #                                  (424.5*rent_2020['fourhundred_449_percent'])+
       #                                 (474.5*rent_2020['fourfifty_499_percent'])+
        #                                (524.5*rent_2020['fivehundred_549_percent'])+
         #                               (574.5*rent_2020['fivefifty_599_percent'])+
          #                              (624.5*rent_2020['sixhundred_649_percent'])+
           #                             (674.5*rent_2020['sixfifty_699_percent'])+
             #                           (724.5*rent_2020['sevenhundred_749_percent'])+
              #                          (774.5*rent_2020['sevenfifty_799_percent'])+
               #                         (849.5*rent_2020['eighthundred_899_percent'])+
                #                        (949.5*rent_2020['ninehundred_999_percent'])+
                 #                       (1124.5*rent_2020['onethousand_1249_percent'])+
                  #                      (1374.5*rent_2020['onethousandtwofifty_1499_percent'])+
                   #                     (1749.5*rent_2020['onethousandfive_1999_percent'])+
                    #                    (2249.5*rent_2020['twothousand_2499_percent'])+
                     #                   (2749.5*rent_2020['twothousandfive_2999_percent'])+
                      #                  (3249.5*rent_2020['threethousand_3499_percent']))

In [928]:
#rent_2020

In [929]:
#rent_2010 = pd.read_csv('ACSDT5Y2010.B25063_data_with_overlays_2022-05-15T164412.csv')
#rent_2010.drop(index=rent_2010.index[0], 
#        axis=0, 
 #       inplace=True)
#rent_2010.columns

In [930]:
#rent_2010 = rent_2010.rename(columns={"B25063_001E": "Total_2010", "B25063_002E": "Total_cash_2010", 
 #                                           "B25063_003E": "less_than_100_2010", "B25063_004E":"onehundred_149_2010", "B25063_005E": "onefifty_199_2010","B25063_006E":"twohundred_249_2010", "B25063_007E":"twofifty_299_2010", "B25063_008E":"threehundred_349_2010","B25063_009E":"threefifty_399_2010", "B25063_010E":"fourhundred_449_2010", "B25063_011E":
  #                                         "fourfifty_499_2010", "B25063_012E":"fivehundred_549_2010", "B25063_013E":"fivefifty_599_2010", "B25063_014E":
   #                                        "sixhundred_649_2010", "B25063_015E":"sixfifty_699_2010", "B25063_016E":"sevenhundred_749_2010", "B25063_017E":"sevenfifty_799_2010",
    #                                       "B25063_018E":"eighthundred_899_2010", "B25063_019E":"ninehundred_999_2010", "B25063_020E":"onethousand_1249_2010", "B25063_021E":"onethousandtwofifty_1499_2010",
     #                                      "B25063_022E":"onethousandfive_1999_2010", "B25063_023E":"twothousandplus_2010", "B25063_024E":"Total_no_cash_2010"})

In [931]:
#rent_2010 = rent_2010.drop(columns=['B25063_001M', 'B25063_002M', 'B25063_003M', 'B25063_004M', 'B25063_005M',
 #                                   'B25063_006M', 'B25063_007M', 'B25063_008M','B25063_009M', 'B25063_010M', 'B25063_011M',
  #                                  'B25063_012M', 'B25063_013M', 'B25063_014M', 'B25063_015M', 'B25063_016M', 'B25063_017M',
   #                                 'B25063_018M', 'B25063_019M', 'B25063_020M', 'B25063_021M', 'B25063_022M', 'B25063_023M',
    #                                'B25063_024M'])

In [932]:
#rent_2010['Total_2010'] = rent_2010.Total_2010.astype(float)
#rent_2010['Total_cash_2010'] = rent_2010.Total_cash_2010.astype(float)
#rent_2010['less_than_100_2010'] = rent_2010.less_than_100_2010.astype(float)
#rent_2010['onehundred_149_2010'] = rent_2010.onehundred_149_2010.astype(float)
#rent_2010['onefifty_199_2010'] = rent_2010.onefifty_199_2010.astype(float)
#rent_2010['twohundred_249_2010'] = rent_2010.twohundred_249_2010.astype(float)
#rent_2010['twofifty_299_2010'] = rent_2010.twofifty_299_2010.astype(float)
#rent_2010['threehundred_349_2010'] = rent_2010.threehundred_349_2010.astype(float)
#rent_2010['threefifty_399_2010'] = rent_2010.threefifty_399_2010.astype(float)
#rent_2010['fourhundred_449_2010'] = rent_2010.fourhundred_449_2010.astype(float)
#rent_2010['fourfifty_499_2010'] = rent_2010.fourfifty_499_2010.astype(float)
#rent_2010['fivehundred_549_2010'] = rent_2010.fivehundred_549_2010.astype(float)
#rent_2010['fivefifty_599_2010'] = rent_2010.fivefifty_599_2010.astype(float)
#rent_2010['sixhundred_649_2010'] = rent_2010.sixhundred_649_2010.astype(float)
#rent_2010['sixfifty_699_2010'] = rent_2010.sixfifty_699_2010.astype(float)
#rent_2010['sevenhundred_749_2010'] = rent_2010.sevenhundred_749_2010.astype(float)
#rent_2010['sevenfifty_799_2010'] = rent_2010.sevenfifty_799_2010.astype(float)
#rent_2010['eighthundred_899_2010'] = rent_2010.eighthundred_899_2010.astype(float)
#rent_2010['ninehundred_999_2010'] = rent_2010.ninehundred_999_2010.astype(float)
#rent_2010['onethousand_1249_2010'] = rent_2010.onethousand_1249_2010.astype(float)
#rent_2010['onethousandtwofifty_1499_2010'] = rent_2010.onethousandtwofifty_1499_2010.astype(float)
#rent_2010['onethousandfive_1999_2010'] = rent_2010.onethousandfive_1999_2010.astype(float)
#rent_2010['twothousandplus_2010'] = rent_2010.twothousandplus_2010.astype(float)
#rent_2010['Total_no_cash_2010'] = rent_2010.Total_no_cash_2010.astype(float)

In [933]:
#rent_2010

In [934]:
#rent_2010['Total_cash_percent_2010'] = rent_2010['Total_cash_2010']/rent_2010['Total_2010']

In [935]:
#rent_2010['less_than_100_percent_2010'] = rent_2010['less_than_100_2010']/rent_2010['Total_cash_2010']

In [936]:
#rent_2010['onehundred_149_percent_2010'] = rent_2010['onehundred_149_2010']/rent_2010['Total_cash_2010']

In [937]:
#rent_2010['onefifty_199_percent_2010'] = rent_2010['onefifty_199_2010']/rent_2010['Total_cash_2010']

In [938]:
#rent_2010['twohundred_249_percent_2010'] = rent_2010['twohundred_249_2010']/rent_2010['Total_cash_2010']

In [939]:
#rent_2010['twofifty_299_percent_2010'] = rent_2010['twofifty_299_2010']/rent_2010['Total_cash_2010']

In [940]:
#rent_2010['threehundred_349_percent_2010'] = rent_2010['threehundred_349_2010']/rent_2010['Total_cash_2010']

In [941]:
#rent_2010['threefifty_399_percent_2010'] = rent_2010['threefifty_399_2010']/rent_2010['Total_cash_2010']

In [942]:
#rent_2010['fourhundred_449_percent_2010'] = rent_2010['fourhundred_449_2010']/rent_2010['Total_cash_2010']

In [943]:
#rent_2010['fourfifty_499_percent_2010'] = rent_2010['fourfifty_499_2010']/rent_2010['Total_cash_2010']

In [944]:
#rent_2010['fivehundred_549_percent_2010'] = rent_2010['fivehundred_549_2010']/rent_2010['Total_cash_2010']

In [945]:
#rent_2010['fivefifty_599_percent_2010'] = rent_2010['fivefifty_599_2010']/rent_2010['Total_cash_2010']

In [946]:
#rent_2010['sixhundred_649_percent_2010'] = rent_2010['sixhundred_649_2010']/rent_2010['Total_cash_2010']

In [947]:
#rent_2010['sixfifty_699_percent_2010'] = rent_2010['sixfifty_699_2010']/rent_2010['Total_cash_2010']
#rent_2010['sevenhundred_749_percent_2010'] = rent_2010['sevenhundred_749_2010']/rent_2010['Total_cash_2010']
#rent_2010['sevenfifty_799_percent_2010'] = rent_2010['sevenfifty_799_2010']/rent_2010['Total_cash_2010']
#rent_2010['eighthundred_899_percent_2010'] = rent_2010['eighthundred_899_2010']/rent_2010['Total_cash_2010']
#rent_2010['ninehundred_999_percent_2010'] = rent_2010['ninehundred_999_2010']/rent_2010['Total_cash_2010']
#rent_2010['onethousand_1249_percent_2010'] = rent_2010['onethousand_1249_2010']/rent_2010['Total_cash_2010']
#rent_2010['onethousandtwofifty_1499_percent_2010'] = rent_2010['onethousandtwofifty_1499_2010']/rent_2010['Total_cash_2010']
#rent_2010['onethousandfive_1999_percent_2010'] = rent_2010['onethousandfive_1999_2010']/rent_2010['Total_cash_2010']
#rent_2010['twothousandplus_percent_2010'] = rent_2010['twothousandplus_2010']/rent_2010['Total_cash_2010']
#rent_2010['Total_no_cash_percent_2010'] = rent_2010['Total_no_cash_2010']/rent_2010['Total_2010']

In [948]:
#rent_2010['avg_rent_2010'] = ((50*rent_2010['less_than_100_percent_2010'])+
 #                                       (124.5*rent_2010['onehundred_149_percent_2010'])+
  #                                      (174.5*rent_2010['onefifty_199_percent_2010'])+
   #                                     (224.5*rent_2010['twohundred_249_percent_2010'])+
    #                                    (274.5*rent_2010['twofifty_299_percent_2010'])+
     #                                   (324.5*rent_2010['threehundred_349_percent_2010'])+
      #                                  (374.5*rent_2010['threefifty_399_percent_2010'])+
       #                                 (424.5*rent_2010['fourhundred_449_percent_2010'])+
        #                                (474.5*rent_2010['fourfifty_499_percent_2010'])+
         #                               (524.5*rent_2010['fivehundred_549_percent_2010'])+
          #                              (574.5*rent_2010['fivefifty_599_percent_2010'])+
           #                             (624.5*rent_2010['sixhundred_649_percent_2010'])+
            #                            (674.5*rent_2010['sixfifty_699_percent_2010'])+
             #                           (724.5*rent_2010['sevenhundred_749_percent_2010'])+
              #                          (774.5*rent_2010['sevenfifty_799_percent_2010'])+
               #                         (849.5*rent_2010['eighthundred_899_percent_2010'])+
                #                        (949.5*rent_2010['ninehundred_999_percent_2010'])+
                 #                       (1124.5*rent_2010['onethousand_1249_percent_2010'])+
                  #                      (1374.5*rent_2010['onethousandtwofifty_1499_percent_2010'])+
                   #                     (1749.5*rent_2010['onethousandfive_1999_percent_2010'])+
                    #                    (2249.5*rent_2010['twothousandplus_percent_2010']))

In [949]:
#rent_2010

In [950]:
#rent_2020['GEOID'] = rent_2020.GEO_ID.str.split('US0', 1, expand = True).get(1)
#rent_2020['GEOID'] = rent_2020.GEOID.str.replace('NA','0').astype(float)

In [951]:
#rent_2020

In [952]:
#rent_2010['GEOID'] = rent_2010.GEO_ID.str.split('US0', 1, expand = True).get(1)
#rent_2010['GEOID'] = rent_2010.GEOID.str.replace('NA','0').astype(float)

In [953]:
#rent_2010

In [954]:
#census_neighborhoods_UDP = final_join_hh_size.merge(
 #   rent_2010, 
  #  on=["GEOID"], 
   # how="left",
    #indicator=True,
#)
#census_neighborhoods_UDP

In [955]:
#census_neighborhoods_UDP.query("_merge1 != 'both'")
#census_neighborhoods_UDP = census_neighborhoods_UDP.rename(columns={"_merge": "_merge1"})

In [956]:
#census_neighborhoods_UDP = census_neighborhoods_UDP.merge(
 #   rent_2020, 
  #  on=["GEOID"], 
   # how="left",
    #indicator=True,
#)
#census_neighborhoods_UDP

In [957]:
#census_neighborhoods_UDP.query("_merge != 'both'")

In [958]:
#census_neighborhoods_UDP = census_neighborhoods_UDP.rename(columns={"_merge": "_merge2"})

In [959]:
#census_neighborhoods_UDP.columns

In [960]:
#list(census_neighborhoods_UDP.columns.values.tolist())

In [961]:
#census_neighborhoods_UDP = census_neighborhoods_UDP.dropna(subset=['NAME_x', 'geoid_x', 'geoid_y', 'name_y', 'geometry_y', 'GEO_ID_x', 'NAME_y','GEO_ID_y'])

In [962]:
#census_neighborhoods_UDP = census_neighborhoods_UDP.rename(columns ={'name_x':'name', 
 #                                                                    'geometry_x':'geometry',
  #                                                                   'Total_2010_x':'Total_2010_hh', 
   #                                                                  'Total_x':'Total_hh', 
    ##                                                                'Total_y':'Total_rent'})

In [963]:
#census_neighborhoods_UDP = census_neighborhoods_UDP.dropna(subset=['avg_rent_2020_x'])

In [964]:
#census_neighborhoods_UDP['rent_percent_change'] = (census_neighborhoods_UDP['avg_rent_2020']-census_neighborhoods_UDP['avg_rent_2010'])/census_neighborhoods_UDP['avg_rent_2010']
#census_neighborhoods_UDP['no_cash_rent_percent_change'] = (census_neighborhoods_UDP['Total_no_cash_2010']-census_neighborhoods_UDP['Total_no_cash'])/census_neighborhoods_UDP['Total_no_cash_2010']

In [965]:
#census_neighborhoods_UDP.rent_percent_change

In [966]:
#census_neighborhoods_UDP['rent_percent_change'].mean()

In [967]:
#census_neighborhoods_UDP['rent_percent_change'].median()

## Census Data - Income

I read in and clean income data for 2020 and 2010.

In [968]:
income_2020 = pd.read_csv('ACSST5Y2020.S1903_data_with_overlays_2022-04-27T190056.csv')
income_2020.drop(index=income_2020.index[0], 
        axis=0, 
        inplace=True)
income_2020.columns

Index(['GEO_ID', 'NAME', 'S1903_C01_001E', 'S1903_C01_001M', 'S1903_C01_002E',
       'S1903_C01_002M', 'S1903_C01_003E', 'S1903_C01_003M', 'S1903_C01_004E',
       'S1903_C01_004M',
       ...
       'S1903_C03_036E', 'S1903_C03_036M', 'S1903_C03_037E', 'S1903_C03_037M',
       'S1903_C03_038E', 'S1903_C03_038M', 'S1903_C03_039E', 'S1903_C03_039M',
       'S1903_C03_040E', 'S1903_C03_040M'],
      dtype='object', length=242)

In [969]:
income_2020 = income_2020.rename(columns={'S1903_C03_034E':'median_income_nonfamily', 
                                          'S1903_C03_015E': 'median_income_family'})

In [970]:
income_2020 = income_2020[['GEO_ID','NAME','median_income_nonfamily', 'median_income_family']]

In [971]:
income_2020.sample(10)

,GEO_ID,NAME,median_income_nonfamily,median_income_family
1864,1400000US06037551502,"Census Tract 5515.02, Los Angeles County, Cali...",37628,88520
1330,1400000US06037407501,"Census Tract 4075.01, Los Angeles County, Cali...",49911,71328
1234,1400000US06037402801,"Census Tract 4028.01, Los Angeles County, Cali...",-,43534
1117,1400000US06037301203,"Census Tract 3012.03, Los Angeles County, Cali...",50313,70826
429,1400000US06037186404,"Census Tract 1864.04, Los Angeles County, Cali...",65188,61723
1620,1400000US06037503201,"Census Tract 5032.01, Los Angeles County, Cali...",-,99423
1062,1400000US06037293306,"Census Tract 2933.06, Los Angeles County, Cali...",50000,109500
1510,1400000US06037480102,"Census Tract 4801.02, Los Angeles County, Cali...",52849,105341
316,1400000US06037134801,"Census Tract 1348.01, Los Angeles County, Cali...",30804,100350
203,1400000US06037123700,"Census Tract 1237, Los Angeles County, California",61393,86297


In [972]:
income_2020['GEOID'] = income_2020.GEO_ID.str.split('US0', 1, expand = True).get(1)
income_2020['GEOID'] = income_2020.GEOID.str.replace('NA','0').astype(float)

In [973]:
income_2010 = pd.read_csv('ACSST5Y2010.S1903_data_with_overlays_2022-05-16T191049.csv')
income_2010.drop(index=income_2010.index[0], 
        axis=0, 
        inplace=True)
income_2010.columns

Index(['GEO_ID', 'S1903_C01_001E', 'S1903_C01_001M', 'S1903_C01_002E',
       'S1903_C01_002M', 'S1903_C01_003E', 'S1903_C01_003M', 'S1903_C01_004E',
       'S1903_C01_004M', 'S1903_C01_005E',
       ...
       'S1903_C02_027M', 'S1903_C02_028E', 'S1903_C02_028M', 'S1903_C02_029E',
       'S1903_C02_029M', 'S1903_C02_030E', 'S1903_C02_030M', 'S1903_C02_031E',
       'S1903_C02_031M', 'NAME'],
      dtype='object', length=122)

In [974]:
income_2010 = income_2010.rename(columns={'S1903_C02_022E':'median_income_nonfamily_2010', 
                                          'S1903_C02_016E': 'median_income_family_2010'})

In [975]:
income_2010 = income_2010[['GEO_ID','NAME','median_income_nonfamily_2010', 'median_income_family_2010']]

In [976]:
income_2010['GEOID'] = income_2010.GEO_ID.str.split('US0', 1, expand = True).get(1)
income_2010['GEOID'] = income_2010.GEOID.str.replace('NA','0').astype(float)

I merge the 2010, and then the 2020, income data to my base dataset from above (which includes the census tracts, neighborhoods, typologies for our neighborhoods of interest, household size percent change, and gross rent percent change with and without inflation). I do some final cleanup on the 133 matched rows and then calculate percent change across the 2 years. 

In [977]:
census_neighborhoods_UDP = census_neighborhoods_UDP.merge(
    income_2020, 
    on=["GEOID"], 
    how="left",
    #indicator=True,
)

In [978]:
census_neighborhoods_UDP.query("_merge1 != 'both'")

,OBJECTID,name,geometry,GEOID,Typology,GEO_ID_x,Total_hhsize_2010,owner_occupied_hh_2010,renter_occupied_hh_2010,_merge1,...,NAME_x,median_gross_rent_2010,rent_percent_change,median_gross_rent_2010_inflation,_merge,rent_percent_change_inflation,GEO_ID,NAME,median_income_nonfamily,median_income_family


In [979]:
census_neighborhoods_UDP = census_neighborhoods_UDP.rename(columns={"_merge": "_merge1"})

In [980]:
census_neighborhoods_UDP = census_neighborhoods_UDP.merge(
    income_2010, 
    on=["GEOID"], 
    how="left",
    indicator=True,
)
census_neighborhoods_UDP

/Users/micaobrien/opt/anaconda3/envs/uds/lib/python3.8/site-packages/geopandas/geodataframe.py:1378: FutureWarning: Passing 'suffixes' which cause duplicate columns {'NAME_x', 'GEO_ID_x'} in the result is deprecated and will raise a MergeError in a future version.
  result = DataFrame.merge(self, *args, **kwargs)


,OBJECTID,name,geometry,GEOID,Typology,GEO_ID_x,Total_hhsize_2010,owner_occupied_hh_2010,renter_occupied_hh_2010,_merge1,...,rent_percent_change_inflation,GEO_ID_x,NAME_x,median_income_nonfamily,median_income_family,GEO_ID_y,NAME_y,median_income_nonfamily_2010,median_income_family_2010,_merge
0,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037210e+09,Advanced Gentrification,1400000US06037210010,2.99,3.20,2.97,both,...,0.120032,1400000US06037210010,"Census Tract 2100.10, Los Angeles County, Cali...",25346,27219,1400000US06037210010,"Census Tract 2100.10, Los Angeles County, Cali...",14800,25536,both
1,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification,1400000US06037209300,2.73,4.47,2.68,both,...,-0.063634,1400000US06037209300,"Census Tract 2093, Los Angeles County, California",24526,34388,1400000US06037209300,"Census Tract 2093, Los Angeles County, California",24286,24982,both
2,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037211e+09,Early/Ongoing Gentrification,1400000US06037211122,2.55,6.60,2.53,both,...,0.063966,1400000US06037211122,"Census Tract 2111.22, Los Angeles County, Cali...",53219,37639,1400000US06037211122,"Census Tract 2111.22, Los Angeles County, Cali...",40036,31622,both
3,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037193e+09,Early/Ongoing Gentrification,1400000US06037192700,2.80,2.60,2.83,both,...,0.163751,1400000US06037192700,"Census Tract 1927, Los Angeles County, California",48950,56129,1400000US06037192700,"Census Tract 1927, Los Angeles County, California",20577,40548,both
4,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,At Risk of Gentrification,1400000US06037208302,2.90,4.32,2.77,both,...,0.003410,1400000US06037208302,"Census Tract 2083.02, Los Angeles County, Cali...",50363,44358,1400000US06037208302,"Census Tract 2083.02, Los Angeles County, Cali...",11818,26368,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037221e+09,Low-Income/Susceptible to Displacement,1400000US06037221120,4.55,7.29,4.10,both,...,-0.122242,1400000US06037221120,"Census Tract 2211.20, Los Angeles County, Cali...",47740,43074,1400000US06037221120,"Census Tract 2211.20, Los Angeles County, Cali...",13333,29928,both
129,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037221e+09,At Risk of Gentrification,1400000US06037221110,3.01,2.96,3.01,both,...,0.113036,1400000US06037221110,"Census Tract 2211.10, Los Angeles County, Cali...",-,35208,1400000US06037221110,"Census Tract 2211.10, Los Angeles County, Cali...",15750,23508,both
130,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,Advanced Gentrification,1400000US06037209820,4.06,5.26,3.78,both,...,-0.076647,1400000US06037209820,"Census Tract 2098.20, Los Angeles County, Cali...",24412,43636,1400000US06037209820,"Census Tract 2098.20, Los Angeles County, Cali...",25438,25841,both
131,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,At Risk of Gentrification,1400000US06037209810,2.53,2.09,2.54,both,...,0.074746,1400000US06037209810,"Census Tract 2098.10, Los Angeles County, Cali...",33563,28618,1400000US06037209810,"Census Tract 2098.10, Los Angeles County, Cali...",23750,24226,both


In [981]:
census_neighborhoods_UDP.query("_merge != 'both'")

,OBJECTID,name,geometry,GEOID,Typology,GEO_ID_x,Total_hhsize_2010,owner_occupied_hh_2010,renter_occupied_hh_2010,_merge1,...,rent_percent_change_inflation,GEO_ID_x,NAME_x,median_income_nonfamily,median_income_family,GEO_ID_y,NAME_y,median_income_nonfamily_2010,median_income_family_2010,_merge


In [982]:
census_neighborhoods_UDP = census_neighborhoods_UDP.dropna(subset=['_merge1', '_merge'])

In [983]:
#census_neighborhoods_UDP.loc[census_neighborhoods_UDP['median_income_family'] != '-']

In [984]:
census_neighborhoods_UDP['median_income_family'] = census_neighborhoods_UDP.median_income_family.str.replace('-','NaN').astype(float)
census_neighborhoods_UDP['median_income_nonfamily'] = census_neighborhoods_UDP.median_income_nonfamily.str.replace('-','NaN').astype(float)
census_neighborhoods_UDP['median_income_family_2010'] = census_neighborhoods_UDP.median_income_family_2010.str.replace('-','NaN').astype(float)
census_neighborhoods_UDP['median_income_nonfamily_2010'] = census_neighborhoods_UDP.median_income_nonfamily_2010.str.replace('-','NaN').astype(float)

In [985]:
census_neighborhoods_UDP['income_family_percent_change'] = (census_neighborhoods_UDP['median_income_family']-census_neighborhoods_UDP['median_income_family_2010'])/census_neighborhoods_UDP['median_income_family_2010']
census_neighborhoods_UDP['income_nonfamily_percent_change'] = (census_neighborhoods_UDP['median_income_nonfamily']-census_neighborhoods_UDP['median_income_nonfamily_2010'])/census_neighborhoods_UDP['median_income_nonfamily_2010']

Without adjusting for inflation, there is a 50% average increase in family income and a 56% percent increae in nonfamily income. These numbers are suspicioulsy high. I will adjust for inflation in the next section and reassess. 

In [986]:
census_neighborhoods_UDP['income_family_percent_change'].mean()

0.5051016453874271

In [987]:
census_neighborhoods_UDP['income_family_percent_change'].median()

0.39313375532476913

In [988]:
census_neighborhoods_UDP['income_nonfamily_percent_change'].mean()

0.5624159045010461

In [989]:
census_neighborhoods_UDP['income_nonfamily_percent_change'].median()

0.30349630464154265

### Adjust for Inflation

To calculate inflation, I divide the median family and nonfamily income (2010) by the employment index for January 2010 and then multiply by the employment index for January 2020. I used the CPI for US City average from this website: https://fred.stlouisfed.org/release/tables?rid=11&eid=51576&od=2020-01-01#. I create new columns and recalculate percent change.

In [990]:
census_neighborhoods_UDP['median_income_family_2010_inflation'] = (census_neighborhoods_UDP
                                                                   ['median_income_family_2010']/111.8)*140.0

In [991]:
census_neighborhoods_UDP['median_income_nonfamily_2010_inflation'] = (census_neighborhoods_UDP
                                                                      ['median_income_nonfamily_2010']/111.8)*140.0

In [992]:
census_neighborhoods_UDP['median_income_family_percent_change_inflation'] = (census_neighborhoods_UDP
                                                                             ['median_income_family']-
                                                                             census_neighborhoods_UDP
                                                                             ['median_income_family_2010_inflation']
                                                                            )/census_neighborhoods_UDP['median_income_family_2010_inflation']

In [993]:
census_neighborhoods_UDP['median_income_nonfamily_percent_change_inflation'] = (census_neighborhoods_UDP
                                                                                ['median_income_nonfamily']-
                                                                                census_neighborhoods_UDP
                                                                                ['median_income_nonfamily_2010_inflation']
                                                                               )/census_neighborhoods_UDP['median_income_nonfamily_2010_inflation']

When I run summary statistics on the inflation-adjusted values, we see average percent change in family income increase by about 20% and nonfamily income by about 25%. These numbers make more sense than the ones above (that did not account for inflation). 

In [994]:
census_neighborhoods_UDP['median_income_family_percent_change_inflation'].mean()

0.20193117110224534

In [995]:
census_neighborhoods_UDP['median_income_family_percent_change_inflation'].median()

0.11251681318078

In [996]:
census_neighborhoods_UDP['median_income_nonfamily_percent_change_inflation'].mean()

0.24770070088012103

In [997]:
census_neighborhoods_UDP['median_income_nonfamily_percent_change_inflation'].median()

0.04093490613517475

## Census Data - Percentage White, Nonhispanic residents

I read in and clean racial demographic data for 2020 and 2010.

In [998]:
white_pop_2020 = pd.read_csv('DECENNIALPL2020.P2_data_with_overlays_2022-05-16T194527.csv')
white_pop_2020.drop(index=white_pop_2020.index[0], 
        axis=0, 
        inplace=True)
white_pop_2020.columns

Index(['GEO_ID', 'NAME', 'P2_001N', 'P2_002N', 'P2_003N', 'P2_004N', 'P2_005N',
       'P2_006N', 'P2_007N', 'P2_008N', 'P2_009N', 'P2_010N', 'P2_011N',
       'P2_012N', 'P2_013N', 'P2_014N', 'P2_015N', 'P2_016N', 'P2_017N',
       'P2_018N', 'P2_019N', 'P2_020N', 'P2_021N', 'P2_022N', 'P2_023N',
       'P2_024N', 'P2_025N', 'P2_026N', 'P2_027N', 'P2_028N', 'P2_029N',
       'P2_030N', 'P2_031N', 'P2_032N', 'P2_033N', 'P2_034N', 'P2_035N',
       'P2_036N', 'P2_037N', 'P2_038N', 'P2_039N', 'P2_040N', 'P2_041N',
       'P2_042N', 'P2_043N', 'P2_044N', 'P2_045N', 'P2_046N', 'P2_047N',
       'P2_048N', 'P2_049N', 'P2_050N', 'P2_051N', 'P2_052N', 'P2_053N',
       'P2_054N', 'P2_055N', 'P2_056N', 'P2_057N', 'P2_058N', 'P2_059N',
       'P2_060N', 'P2_061N', 'P2_062N', 'P2_063N', 'P2_064N', 'P2_065N',
       'P2_066N', 'P2_067N', 'P2_068N', 'P2_069N', 'P2_070N', 'P2_071N',
       'P2_072N', 'P2_073N'],
      dtype='object')

In [999]:
white_pop_2020 = white_pop_2020.rename(columns={'P2_001N':'total_race', 
                                          'P2_005N': 'nonhispanic_white'})

In [1000]:
white_pop_2020 = white_pop_2020[['GEO_ID','NAME','total_race', 'nonhispanic_white']]

In [1001]:
white_pop_2020['total_race'] = white_pop_2020.total_race.astype(float)
white_pop_2020['nonhispanic_white'] = white_pop_2020.nonhispanic_white.astype(float)

In [1002]:
white_pop_2020['percent_white']= white_pop_2020['nonhispanic_white']/white_pop_2020['total_race']

In [1003]:
white_pop_2020['GEOID'] = white_pop_2020.GEO_ID.str.split('US0', 1, expand = True).get(1)
white_pop_2020['GEOID'] = white_pop_2020.GEOID.str.replace('NA','0').astype(float)

In [1004]:
white_pop_2010 = pd.read_csv('DECENNIALPL2010.P2_data_with_overlays_2022-05-16T200232.csv')
white_pop_2010.drop(index=white_pop_2010.index[0], 
        axis=0, 
        inplace=True)
white_pop_2010.columns

Index(['P002050', 'P002051', 'P002052', 'P002053', 'P002054', 'P002055',
       'P002056', 'P002057', 'P002058', 'P002059', 'P002060', 'P002061',
       'P002062', 'P002063', 'P002064', 'P002065', 'P002066', 'P002067',
       'P002068', 'P002069', 'P002070', 'P002071', 'P002072', 'P002073',
       'P002049', 'GEO_ID', 'P002001', 'P002002', 'P002003', 'P002004',
       'P002005', 'P002006', 'P002007', 'P002008', 'P002009', 'P002010',
       'P002011', 'P002012', 'P002013', 'P002014', 'P002015', 'P002016',
       'P002017', 'P002018', 'P002019', 'P002020', 'P002021', 'P002022',
       'P002023', 'P002024', 'P002025', 'P002026', 'P002027', 'P002028',
       'P002029', 'P002030', 'P002031', 'P002032', 'P002033', 'P002034',
       'P002035', 'P002036', 'P002037', 'P002038', 'P002039', 'P002040',
       'P002041', 'P002042', 'P002043', 'P002044', 'P002045', 'P002046',
       'P002047', 'P002048', 'NAME'],
      dtype='object')

In [1005]:
white_pop_2010 = white_pop_2010.rename(columns={'P002001':'total_race_2010', 
                                          'P002005': 'nonhispanic_white_2010'})

In [1006]:
white_pop_2010 = white_pop_2010[['GEO_ID','NAME','total_race_2010', 'nonhispanic_white_2010']]
white_pop_2010.sample(10)

,GEO_ID,NAME,total_race_2010,nonhispanic_white_2010
2326,1400000US06037980009,"Census Tract 9800.09, Los Angeles County, Cali...",14,8
651,1400000US06037218300,"Census Tract 2183, Los Angeles County, California",5520,374
2065,1400000US06037650603,"Census Tract 6506.03, Los Angeles County, Cali...",3958,1926
784,1400000US06037239201,"Census Tract 2392.01, Los Angeles County, Cali...",3562,25
1033,1400000US06037310202,"Census Tract 3102.02, Los Angeles County, Cali...",4985,3565
593,1400000US06037211910,"Census Tract 2119.10, Los Angeles County, Cali...",4657,345
375,1400000US06037185100,"Census Tract 1851, Los Angeles County, California",6295,1992
2258,1400000US06037920015,"Census Tract 9200.15, Los Angeles County, Cali...",6353,4150
793,1400000US06037239701,"Census Tract 2397.01, Los Angeles County, Cali...",4261,28
1688,1400000US06037542401,"Census Tract 5424.01, Los Angeles County, Cali...",4912,38


In [1007]:
white_pop_2010.loc[white_pop_2010['total_race_2010'] == '4875(r39144)']

,GEO_ID,NAME,total_race_2010,nonhispanic_white_2010
1792,1400000US06037554406,"Census Tract 5544.06, Los Angeles County, Cali...",4875(r39144),1906


In [1008]:
white_pop_2010 = white_pop_2010.replace({'4875(r39144)': '4875'})
white_pop_2010 = white_pop_2010.replace({'3703(r39145)': '3703'})
white_pop_2010 = white_pop_2010.replace({'3902(r39146)': '3902'})
white_pop_2010 = white_pop_2010.replace({'4883(r39147)': '4883'})
white_pop_2010 = white_pop_2010.replace({'3613(r39148)': '3613'})
white_pop_2010 = white_pop_2010.replace({'5752(r39149)': '5752'})
white_pop_2010 = white_pop_2010.replace({'4787(r39150)': '4787'})

In [1009]:
white_pop_2010['total_race_2010'] = white_pop_2010.total_race_2010.astype(float)
white_pop_2010['nonhispanic_white_2010'] = white_pop_2010.nonhispanic_white_2010.astype(float)

In [1010]:
white_pop_2010['percent_white_2010']= white_pop_2010['nonhispanic_white_2010']/white_pop_2010['total_race_2010']

In [1011]:
white_pop_2010['GEOID'] = white_pop_2010.GEO_ID.str.split('US0', 1, expand = True).get(1)
white_pop_2010['GEOID'] = white_pop_2010.GEOID.str.replace('NA','0').astype(float)

In [1012]:
white_pop_2020 = white_pop_2020.rename(columns={'GEO_ID':'GEO_ID_white_pop', 
                                          'NAME': 'NAME_white_pop'})

I merge the 2020, and then the 2010, percent white population data to my base dataset from above (which includes the census tracts, neighborhoods, typologies for our neighborhoods of interest, household size percent change, gross rent percent change (with and without inflation), and median income percent change(with and without inflation)). I do some final cleanup on the 133 matched rows and then calculate percent change across the 2 years. 

In [1013]:
census_neighborhoods_UDP = census_neighborhoods_UDP.merge(
    white_pop_2020, 
    on=["GEOID"], 
    how="left",
    #indicator=True,
)

In [1014]:
census_neighborhoods_UDP = census_neighborhoods_UDP.dropna(subset=['GEO_ID_white_pop', 'NAME_white_pop'])

In [1015]:
census_neighborhoods_UDP = census_neighborhoods_UDP.merge(
    white_pop_2010, 
    on=["GEOID"], 
    how="left",
    #indicator=True,
)
census_neighborhoods_UDP

,OBJECTID,name,geometry,GEOID,Typology,GEO_ID_x,Total_hhsize_2010,owner_occupied_hh_2010,renter_occupied_hh_2010,_merge1,...,GEO_ID_white_pop,NAME_white_pop,total_race,nonhispanic_white,percent_white,GEO_ID,NAME,total_race_2010,nonhispanic_white_2010,percent_white_2010
0,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037210e+09,Advanced Gentrification,1400000US06037210010,2.99,3.20,2.97,both,...,1400000US06037210010,"Census Tract 2100.10, Los Angeles County, Cali...",3190.0,130.0,0.040752,1400000US06037210010,"Census Tract 2100.10, Los Angeles County, Cali...",3520.0,87.0,0.024716
1,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification,1400000US06037209300,2.73,4.47,2.68,both,...,1400000US06037209300,"Census Tract 2093, Los Angeles County, California",4712.0,582.0,0.123514,1400000US06037209300,"Census Tract 2093, Los Angeles County, California",5021.0,710.0,0.141406
2,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037211e+09,Early/Ongoing Gentrification,1400000US06037211122,2.55,6.60,2.53,both,...,1400000US06037211122,"Census Tract 2111.22, Los Angeles County, Cali...",2877.0,551.0,0.191519,1400000US06037211122,"Census Tract 2111.22, Los Angeles County, Cali...",3075.0,310.0,0.100813
3,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037193e+09,Early/Ongoing Gentrification,1400000US06037192700,2.80,2.60,2.83,both,...,1400000US06037192700,"Census Tract 1927, Los Angeles County, California",3550.0,545.0,0.153521,1400000US06037192700,"Census Tract 1927, Los Angeles County, California",3513.0,349.0,0.099345
4,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,At Risk of Gentrification,1400000US06037208302,2.90,4.32,2.77,both,...,1400000US06037208302,"Census Tract 2083.02, Los Angeles County, Cali...",4560.0,374.0,0.082018,1400000US06037208302,"Census Tract 2083.02, Los Angeles County, Cali...",4360.0,181.0,0.041514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037221e+09,Low-Income/Susceptible to Displacement,1400000US06037221120,4.55,7.29,4.10,both,...,1400000US06037221120,"Census Tract 2211.20, Los Angeles County, Cali...",3030.0,37.0,0.012211,1400000US06037221120,"Census Tract 2211.20, Los Angeles County, Cali...",3373.0,70.0,0.020753
129,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037221e+09,At Risk of Gentrification,1400000US06037221110,3.01,2.96,3.01,both,...,1400000US06037221110,"Census Tract 2211.10, Los Angeles County, Cali...",3289.0,102.0,0.031012,1400000US06037221110,"Census Tract 2211.10, Los Angeles County, Cali...",3565.0,87.0,0.024404
130,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,Advanced Gentrification,1400000US06037209820,4.06,5.26,3.78,both,...,1400000US06037209820,"Census Tract 2098.20, Los Angeles County, Cali...",2732.0,109.0,0.039898,1400000US06037209820,"Census Tract 2098.20, Los Angeles County, Cali...",2968.0,59.0,0.019879
131,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,At Risk of Gentrification,1400000US06037209810,2.53,2.09,2.54,both,...,1400000US06037209810,"Census Tract 2098.10, Los Angeles County, Cali...",2385.0,217.0,0.090985,1400000US06037209810,"Census Tract 2098.10, Los Angeles County, Cali...",2472.0,183.0,0.074029


In [1016]:
census_neighborhoods_UDP = census_neighborhoods_UDP.dropna(subset=['GEO_ID', 'NAME'])

In [1017]:
census_neighborhoods_UDP['white_pop_percent_change'] = (census_neighborhoods_UDP['percent_white']-
                                                            census_neighborhoods_UDP['percent_white_2010']
                                                       )/census_neighborhoods_UDP['percent_white_2010']

In [1018]:
census_neighborhoods_UDP.white_pop_percent_change

0      0.648831
1     -0.126527
2      0.899744
3      0.545329
4      0.975671
         ...   
128   -0.411594
129    0.270798
130    1.007048
131    0.229048
132    1.656966
Name: white_pop_percent_change, Length: 133, dtype: float64

I run summary statistics and find a 66% average increase in the percent of white people living in our neighborhoods of intererst. I found this to be the most interesting outcome so far and could be a sign of advanced gentrification. Then again, when we consider that many of these neighborhoods had very small white populations in 2010, any small increase could result in a large percent change. This is something to explore with future research. 

In [1019]:
census_neighborhoods_UDP['white_pop_percent_change'].mean()

0.6610114361481213

In [1020]:
census_neighborhoods_UDP['white_pop_percent_change'].median()

0.4927357775055168

## Census Data - Percent of residents 25 and older with Bachelors Degree or higher

Lastly, I read in and clean educational attainment data for 2020 and 2010.

In [1021]:
ed_level_2020 = pd.read_csv('ACSST5Y2020.S1501_data_with_overlays_2022-05-17T211357.csv')
ed_level_2020.drop(index=ed_level_2020.index[0], 
        axis=0, 
        inplace=True)
ed_level_2020.columns

/var/folders/bs/d9h__8tn0rb0h3qzd8bn64380000gn/T/ipykernel_16310/3456229734.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,162,163,164,165,174,175,176,177,186,187,188,189,234,235,236,237,240,241,244,245,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,3

Index(['GEO_ID', 'NAME', 'S1501_C01_001E', 'S1501_C01_001M', 'S1501_C01_002E',
       'S1501_C01_002M', 'S1501_C01_003E', 'S1501_C01_003M', 'S1501_C01_004E',
       'S1501_C01_004M',
       ...
       'S1501_C06_060E', 'S1501_C06_060M', 'S1501_C06_061E', 'S1501_C06_061M',
       'S1501_C06_062E', 'S1501_C06_062M', 'S1501_C06_063E', 'S1501_C06_063M',
       'S1501_C06_064E', 'S1501_C06_064M'],
      dtype='object', length=770)

In [1022]:
ed_level_2020 = ed_level_2020[['S1501_C01_015E','NAME','GEO_ID', 'S1501_C01_006E']]

In [1023]:
ed_level_2020 = ed_level_2020.rename(columns={'S1501_C01_015E':'twentyfive_over_with_bachelorsplus', 
                                          'S1501_C01_006E': 'total_pop_25_over'})

In [1024]:
ed_level_2020['twentyfive_over_with_bachelorsplus'] = ed_level_2020.twentyfive_over_with_bachelorsplus.astype(float)
ed_level_2020['total_pop_25_over'] = ed_level_2020.total_pop_25_over.astype(float)

In [1025]:
ed_level_2020['percent_with_bachelorsplus'] = ed_level_2020['twentyfive_over_with_bachelorsplus']/ed_level_2020['total_pop_25_over']

In [1026]:
ed_level_2020['GEOID'] = ed_level_2020.GEO_ID.str.split('US0', 1, expand = True).get(1)
ed_level_2020['GEOID'] = ed_level_2020.GEOID.str.replace('NA','0').astype(float)

In [1027]:
ed_level_2010 = pd.read_csv('ACSST5Y2010.S1501_data_with_overlays_2022-05-17T213422.csv')
ed_level_2010.drop(index=ed_level_2020.index[0], 
        axis=0, 
        inplace=True)
ed_level_2010.columns

Index(['GEO_ID', 'S1501_C01_001E', 'S1501_C01_001M', 'S1501_C01_002E',
       'S1501_C01_002M', 'S1501_C01_003E', 'S1501_C01_003M', 'S1501_C01_004E',
       'S1501_C01_004M', 'S1501_C01_005E',
       ...
       'S1501_C03_034M', 'S1501_C03_035E', 'S1501_C03_035M', 'S1501_C03_036E',
       'S1501_C03_036M', 'S1501_C03_037E', 'S1501_C03_037M', 'S1501_C03_038E',
       'S1501_C03_038M', 'NAME'],
      dtype='object', length=230)

In [1028]:
ed_level_2010 = ed_level_2010[['S1501_C01_015E','NAME','GEO_ID', 'S1501_C01_006E']]

In [1029]:
ed_level_2010 = ed_level_2010.rename(columns={'S1501_C01_015E':'percent_with_bachelorsplus_2010', 
                                          'S1501_C01_006E': 'total_pop_25_over_2010'})

In [1030]:
ed_level_2010 = ed_level_2010.iloc[1: , :]
ed_level_2010

,percent_with_bachelorsplus_2010,NAME,GEO_ID,total_pop_25_over_2010
2,35.7,"Census Tract 1011.22, Los Angeles County, Cali...",1400000US06037101122,2651
3,13.6,"Census Tract 1012.10, Los Angeles County, Cali...",1400000US06037101210,4572
4,17.0,"Census Tract 1012.20, Los Angeles County, Cali...",1400000US06037101220,2244
5,33.6,"Census Tract 1013, Los Angeles County, California",1400000US06037101300,2730
6,36.6,"Census Tract 1014, Los Angeles County, California",1400000US06037101400,2817
...,...,...,...,...
2343,-,"Census Tract 9800.33, Los Angeles County, Cali...",1400000US06037980033,0
2344,-,"Census Tract 9901, Los Angeles County, California",1400000US06037990100,0
2345,-,"Census Tract 9902, Los Angeles County, California",1400000US06037990200,0
2346,-,"Census Tract 9903, Los Angeles County, California",1400000US06037990300,0


In [1031]:
ed_level_2010 = ed_level_2010.iloc[:-1 , :]

In [1033]:
ed_level_2010['percent_with_bachelorsplus_2010'] = ed_level_2010.percent_with_bachelorsplus_2010.str.replace('-','NaN').astype(float)
ed_level_2010['total_pop_25_over_2010'] = ed_level_2010.total_pop_25_over_2010.astype(float)

/var/folders/bs/d9h__8tn0rb0h3qzd8bn64380000gn/T/ipykernel_16310/1719286478.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ed_level_2010['percent_with_bachelorsplus_2010'] = ed_level_2010.percent_with_bachelorsplus_2010.str.replace('-','NaN').astype(float)
/var/folders/bs/d9h__8tn0rb0h3qzd8bn64380000gn/T/ipykernel_16310/1719286478.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ed_level_2010['total_pop_25_over_2010'] = ed_level_2010.total_pop_25_over_2010.astype(float)


In [1034]:
ed_level_2010['GEOID'] = ed_level_2010.GEO_ID.str.split('US0', 1, expand = True).get(1)
ed_level_2010['GEOID'] = ed_level_2010.GEOID.str.replace('NA','0').astype(float)

/var/folders/bs/d9h__8tn0rb0h3qzd8bn64380000gn/T/ipykernel_16310/1626662653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ed_level_2010['GEOID'] = ed_level_2010.GEO_ID.str.split('US0', 1, expand = True).get(1)
/var/folders/bs/d9h__8tn0rb0h3qzd8bn64380000gn/T/ipykernel_16310/1626662653.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ed_level_2010['GEOID'] = ed_level_2010.GEOID.str.replace('NA','0').astype(float)


I merge the 2020, and then the 2010, percent with bachelors+ data to my base dataset from above (which includes the census tracts, neighborhoods, typologies for our neighborhoods of interest, household size percent change, gross rent percent change (with and without inflation), median income percent change(with and without inflation), and white population percent change). I do some final cleanup on the 133 matched rows and then calculate percent change across the 2 years. 

In [1035]:
census_neighborhoods_UDP = census_neighborhoods_UDP.merge(
    ed_level_2020, 
    on=["GEOID"], 
    how="left",
    #indicator=True,
)

/Users/micaobrien/opt/anaconda3/envs/uds/lib/python3.8/site-packages/geopandas/geodataframe.py:1378: FutureWarning: Passing 'suffixes' which cause duplicate columns {'NAME_x', 'GEO_ID_x'} in the result is deprecated and will raise a MergeError in a future version.
  result = DataFrame.merge(self, *args, **kwargs)


In [1036]:
census_neighborhoods_UDP = census_neighborhoods_UDP.drop(columns=['GEO_ID_y', 'NAME_y'])

In [1037]:
census_neighborhoods_UDP = census_neighborhoods_UDP.merge(
    ed_level_2010, 
    on=["GEOID"], 
    how="left",
    #indicator=True,
)
census_neighborhoods_UDP

,OBJECTID,name,geometry,GEOID,Typology,GEO_ID_x,Total_hhsize_2010,owner_occupied_hh_2010,renter_occupied_hh_2010,_merge1,...,nonhispanic_white_2010,percent_white_2010,white_pop_percent_change,twentyfive_over_with_bachelorsplus,total_pop_25_over,percent_with_bachelorsplus,percent_with_bachelorsplus_2010,NAME,GEO_ID,total_pop_25_over_2010
0,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037210e+09,Advanced Gentrification,1400000US06037210010,2.99,3.20,2.97,both,...,87.0,0.024716,0.648831,406.0,2254.0,0.180124,9.4,"Census Tract 2100.10, Los Angeles County, Cali...",1400000US06037210010,1981.0
1,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037209e+09,Early/Ongoing Gentrification,1400000US06037209300,2.73,4.47,2.68,both,...,710.0,0.141406,-0.126527,692.0,3387.0,0.204311,20.2,"Census Tract 2093, Los Angeles County, California",1400000US06037209300,2588.0
2,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037211e+09,Early/Ongoing Gentrification,1400000US06037211122,2.55,6.60,2.53,both,...,310.0,0.100813,0.899744,800.0,2178.0,0.367309,33.0,"Census Tract 2111.22, Los Angeles County, Cali...",1400000US06037211122,1981.0
3,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037193e+09,Early/Ongoing Gentrification,1400000US06037192700,2.80,2.60,2.83,both,...,349.0,0.099345,0.545329,639.0,2665.0,0.239775,15.2,"Census Tract 1927, Los Angeles County, California",1400000US06037192700,2257.0
4,107,Westlake,"POLYGON ((-118.25908 34.05389, -118.25984 34.0...",6.037208e+09,At Risk of Gentrification,1400000US06037208302,2.90,4.32,2.77,both,...,181.0,0.041514,0.975671,790.0,3503.0,0.225521,13.0,"Census Tract 2083.02, Los Angeles County, Cali...",1400000US06037208302,2523.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037221e+09,Low-Income/Susceptible to Displacement,1400000US06037221120,4.55,7.29,4.10,both,...,70.0,0.020753,-0.411594,122.0,1858.0,0.065662,4.2,"Census Tract 2211.20, Los Angeles County, Cali...",1400000US06037221120,2039.0
129,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037221e+09,At Risk of Gentrification,1400000US06037221110,3.01,2.96,3.01,both,...,87.0,0.024404,0.270798,192.0,2482.0,0.077357,4.7,"Census Tract 2211.10, Los Angeles County, Cali...",1400000US06037221110,2162.0
130,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,Advanced Gentrification,1400000US06037209820,4.06,5.26,3.78,both,...,59.0,0.019879,1.007048,164.0,1758.0,0.093288,4.9,"Census Tract 2098.20, Los Angeles County, Cali...",1400000US06037209820,1779.0
131,74,Pico-Union,"POLYGON ((-118.28146 34.05250, -118.28094 34.0...",6.037210e+09,At Risk of Gentrification,1400000US06037209810,2.53,2.09,2.54,both,...,183.0,0.074029,0.229048,229.0,1955.0,0.117136,11.3,"Census Tract 2098.10, Los Angeles County, Cali...",1400000US06037209810,2277.0


In [1038]:
census_neighborhoods_UDP = census_neighborhoods_UDP.drop(columns=['GEO_ID', 'NAME'])

In [1039]:
census_neighborhoods_UDP['percent_with_bachelorsplus_2010'] = census_neighborhoods_UDP['percent_with_bachelorsplus_2010']/100

In [1040]:
census_neighborhoods_UDP['bachelorsplus_percent_change'] = (census_neighborhoods_UDP['percent_with_bachelorsplus']-
                                                            census_neighborhoods_UDP['percent_with_bachelorsplus_2010']
                                                           )/census_neighborhoods_UDP['percent_with_bachelorsplus_2010']

In [1041]:
census_neighborhoods_UDP.bachelorsplus_percent_change

0      0.916215
1      0.011439
2      0.113059
3      0.577466
4      0.734777
         ...   
128    0.563381
129    0.645893
130    0.903833
131    0.036598
132    1.113294
Name: bachelorsplus_percent_change, Length: 133, dtype: float64

I calculate summary statistics and see that, like with percent change in white population, the increase in percent of the population with a bachelor's degree and above increases dramatically. We see a 58% average increase in percent of the population with an advanced degree. 

In [1042]:
census_neighborhoods_UDP['bachelorsplus_percent_change'].mean()

0.5752759248103542

In [1043]:
census_neighborhoods_UDP['bachelorsplus_percent_change'].median()

0.4118908614321459

## Cleaning final dataset

I end by deleting extraneous columns and saving two versions of the final dataset as csv files. I save one version with all of the census variables I uses and a second with just the most important variables for clustering.  

In [1044]:
print(census_neighborhoods_UDP.columns.tolist())

['OBJECTID', 'name', 'geometry', 'GEOID', 'Typology', 'GEO_ID_x', 'Total_hhsize_2010', 'owner_occupied_hh_2010', 'renter_occupied_hh_2010', '_merge1', 'Total_hhsize', 'owner_occupied_hh', 'renter_occupied_hh', 'Total_hh_percent_change', 'owner_occupied_percent_change', 'renter_occupied_percent_change', 'median_gross_rent', 'NAME_x', 'median_gross_rent_2010', 'rent_percent_change', 'median_gross_rent_2010_inflation', '_merge1', 'rent_percent_change_inflation', 'GEO_ID_x', 'NAME_x', 'median_income_nonfamily', 'median_income_family', 'median_income_nonfamily_2010', 'median_income_family_2010', '_merge', 'income_family_percent_change', 'income_nonfamily_percent_change', 'median_income_family_2010_inflation', 'median_income_nonfamily_2010_inflation', 'median_income_family_percent_change_inflation', 'median_income_nonfamily_percent_change_inflation', 'GEO_ID_white_pop', 'NAME_white_pop', 'total_race', 'nonhispanic_white', 'percent_white', 'GEO_ID_x', 'NAME_x', 'total_race_2010', 'nonhispanic

In [1045]:
census_neighborhoods_UDP = census_neighborhoods_UDP.drop(columns=['GEO_ID_x'])

In [1046]:
census_neighborhoods_UDP = census_neighborhoods_UDP.drop(columns=['_merge1'])

In [1047]:
census_neighborhoods_UDP = census_neighborhoods_UDP.drop(columns=['NAME_x'])

In [1048]:
census_neighborhoods_UDP = census_neighborhoods_UDP.drop(columns=['_merge'])

In [1049]:
census_neighborhoods_UDP = census_neighborhoods_UDP.drop(columns=['GEO_ID_white_pop'])

In [1050]:
census_neighborhoods_UDP = census_neighborhoods_UDP.drop(columns=['NAME_white_pop'])

In [1051]:
print(census_neighborhoods_UDP.columns.tolist())

['OBJECTID', 'name', 'geometry', 'GEOID', 'Typology', 'Total_hhsize_2010', 'owner_occupied_hh_2010', 'renter_occupied_hh_2010', 'Total_hhsize', 'owner_occupied_hh', 'renter_occupied_hh', 'Total_hh_percent_change', 'owner_occupied_percent_change', 'renter_occupied_percent_change', 'median_gross_rent', 'median_gross_rent_2010', 'rent_percent_change', 'median_gross_rent_2010_inflation', 'rent_percent_change_inflation', 'median_income_nonfamily', 'median_income_family', 'median_income_nonfamily_2010', 'median_income_family_2010', 'income_family_percent_change', 'income_nonfamily_percent_change', 'median_income_family_2010_inflation', 'median_income_nonfamily_2010_inflation', 'median_income_family_percent_change_inflation', 'median_income_nonfamily_percent_change_inflation', 'total_race', 'nonhispanic_white', 'percent_white', 'total_race_2010', 'nonhispanic_white_2010', 'percent_white_2010', 'white_pop_percent_change', 'twentyfive_over_with_bachelorsplus', 'total_pop_25_over', 'percent_with

In [1052]:
#census_neighborhoods_UDP = census_neighborhoods_UDP.rename(columns={"name_x":"name", 'geometry_x':'geometry'})

In [1053]:
census_neighborhoods_UDP.to_csv('Census_neighborhoods_UDP_full.csv')

In [1054]:
census_neighborhoods_UDP_condensed = census_neighborhoods_UDP[['name', 'geometry', 'GEOID', 'Typology', 
                                                               'Total_hh_percent_change',
                                                              'rent_percent_change', 'rent_percent_change_inflation',
                                                              'income_family_percent_change', 'income_nonfamily_percent_change',
                                                              'median_income_family_percent_change_inflation',
                                                               'median_income_nonfamily_percent_change_inflation', 'white_pop_percent_change',
                                                              'bachelorsplus_percent_change']]

In [1055]:
census_neighborhoods_UDP_condensed.to_csv('Census_neighborhoods_UDP_condensed.csv')